<h1>Predict tracrRNAs</h1>
<ol>
    <li><a id="libraries">Load Libraries</a></li>
    <li><a href="#blast">BLAST the consensus repeat of a CRISPR array</a></li>
    <li><a href="#round0Prep">Round 0</a></li>
    <li><a href="#round0Results">Round 0 - Results</a></li>
    <li><a href="#round1Prep">Round 1</a></li>
    <li><a href="#round1Results">Round 1 - Results</a></li>
    <li><a href="#round2Prep">Round 2</a></li>
    <li><a href="#round2Results">Round 2 - Results</a></li>
    <li><a href="#combinedResults">Combined Results</a></li>
    <li><a href="#everything">Extracting Results</a></li>   
</ol>

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

# import tempfile
from sys import path as spath
spath.append("scripts/")

from Bio.Seq import Seq
from Bio.SeqFeature import SeqFeature, FeatureLocation
from Bio.SeqIO import index as fasta_index, parse, write
from Bio.SeqRecord import SeqRecord
from CRISPRtools import * #MakeFasta, PilerCRReader, MinCEDReader
from easyFunctions import BLAST_short, Coordinate, dump
from InfernalResults import *
from HMMParser import *
from os import chdir, path, stat, system
from pandas import Series
from pickle import load
#from Rho import *
# from RhoTermPredict import RhoTermPredict
from easyFunctions import dump

################################         Data          ################################
print("Loaded libraries")
chdir("/mnt/research/germs/shane/transActRNA/data")
gene = "Cas9"
casRelatedProteins = fasta_index("proteins/%s-Like-clustered.faa" % (gene),"fasta")
print("Number of %ss: %s" % (gene,comma(len(casRelatedProteins))))
breakPoints = set(range(100,len(casRelatedProteins),100))
print("Loading Operons")
casOperons = load(open("pickles/%s_Operons.p" % gene,"rb"))
print("Ready to rock and roll!")

Loaded libraries
Number of Cas9s: 2,147
Loading Operons
Ready to rock and roll!


<h2><a id="annotate">Annotate</a></h2>

[Ref](#ref)

In [ ]:
from types import MethodType
operon.annotate = MethodType(annotate,operon)
# infernalResults = ProcessInfernal(0,gene)
operon.annotate(protID,infernalResults.seqTracrs[protID])

counter = 0
for protID in infernalResults.seqTracrs:
    operon = casOperons.operons[casOperons.seqMap[protID]]
    crispr = operon.getCRISPR(protID)
    crispr.repeatSeqs(protID,open(operon.getRepeatPath(protID),'w'))
    blastResults = parseBLAST(BLAST_short(operon.getRepeatPath(protID), operon.getFastaPointer(protID), "blastout/conRepeats/%s.xml" % (protID)))
    if len(blastResults) == 0: continue
    crispr.clusterBLASTResults(blastResults)
    if len(crispr.antiRepeats) == 0: print("No anti's in %s?"%(protID))
    crispr.terminators = []
    crispr.getAntiRepeatCandidates(open("tmp/possibleTracrs.fasta","w"), operon.getSeq())
    res = system("~/bin/Arnold/erpin ~/bin/Arnold/rho-indep.epn tmp/possibleTracrs.fasta -1,4 -add 1 4 2 -pcw 3.0 -cutoff 100% >tmp/rhoInd.out")
    erpOut = ErpinOut()
    crispr.getTracrRNA_Candidates(erpOut,open("test.txt",'w'))
    operon.annotate = MethodType(annotate,operon)
    retVal = operon.annotate(protID,infernalResults.seqTracrs[protID]) 
    counter += int(retVal is not None)
counter
s1="ggaagtctatcagggagttaggtaactgattcccagc".upper()
s2="aacgcgagtgtagcttaatggtaaagcctctgccttccaagcagatgacgcgggttcgattcccgtcactcgc".upper()[:len(s1)+5]
print(calcPercIdent(s1,s2))
alignSequence(s2,s1)

[Home](#libraries)

<h2><a id="blast">BLAST</a> for the consensus repeat, look for termination signals from results, annotate each system </h2>

In [4]:
noPredictedTracr = {} # {ID:Reason it didn't have a tracr}
totalSols, erpSols, breakCount, hadToGetSeq = 0, 0, 0, 0
possibleSol = open("sequences/temp%s_predictedTracrRNAs.fasta" % (gene),"w")
for i, protID in enumerate(casRelatedProteins): #     protID = 'NFJO01000020_ORF977'
    # Step 1. Get the CRISPR operon = casOperons[protID]
    operon = casOperons.operons[casOperons.seqMap[protID]]
    crispr = operon.getCRISPR(protID)
    #crispr.repeatSeqs = MethodType(repeatSeqs,crispr)
    
    # Step 2. Write all consensus repeats to a file
    crispr.repeatSeqs(protID,open(operon.getRepeatPath(protID),'w'))

    # Step 3. Blast the consensus repeats against the Cas-like protein-containing chromosome
    blastResults = parseBLAST(BLAST_short(operon.getRepeatPath(protID), operon.getFastaPointer(protID), "blastout/conRepeats/%s.xml" % (protID)))
    if len(blastResults) == 0: 
        noPredictedTracr[protID] = 'No BLAST results'
        continue
    
    #Step 4. Narrow the blast results down by removing crRNAs
    crispr.clusterBLASTResults(blastResults)
     
    # Step 5. Get the approriate flanking sequence for each anti-repeat candidate
    if len(crispr.antiRepeats) == 0:
        noPredictedTracr[protID] = 'No Anti-repeats'
        continue
    if operon.seq is None: hadToGetSeq += 1; operon.seq = fasta_index(operon.getFastaPointer(protID),'fasta')[protID]

    crispr.getAntiRepeatCandidates(open("tmp/possibleTracrs.fasta","w"), operon.getSeq())
    
    # Step 6. Look for termination signals
    res = system("~/bin/Arnold/erpin ~/bin/Arnold/rho-indep.epn tmp/possibleTracrs.fasta -1,4 -add 1 4 2 -pcw 3.0 -cutoff 100% >tmp/rhoInd.out")
    if res != 0:
        noPredictedTracr[protID] = 'Erpin Failed %i' % (res)
        continue
    
    # Step 7. Read the termination signals
    erpOut = ErpinOut()
    erpSols += len(erpOut.terminators)

    # Step 8. Get tracrRNA candidates with rho-ind signals
    numNewTracrs = crispr.getTracrRNA_Candidates(erpOut,possibleSol)
    if numNewTracrs == 0: noPredictedTracr[protID] = 'No terminators'
    
    # Keep track of how many solutions have been found so far and print
    totalSols += numNewTracrs
    if i in breakPoints: print(i,end=' ')
    if i >50: break

possibleSol.close()
#dump(casOperons, "pickles/%s_Operons.p" % gene)
#dump(noPredictedTracr, "pickles/%sWithNoPredictedTracr.p" % (gene))
print("\nErpin Solutions:", erpSols)
print("Found %i possible tracr solutions from %i assmeblies" % (totalSols,len(casRelatedProteins)-len(noPredictedTracr)))

Failed to create blastDB assemblies/pseudoChromos/AAAHNU010000003.1_ORF209.fasta


TypeError: 'NoneType' object is not iterable

<h2><a id="round0Prep">Round 0 - Predictions From Scratch</a></h2>

In [ ]:
%%bash -s $gene
cd-hit-est -i sequences/$1_predictedTracrRNAs.fasta -o sequences/$1_predictedTracrRNAs.grouped.fasta -M 0 -d 0 -c .85 -T 0 -s .90 -sc 1 >logs/$1_tracrClusterLog.log
tail -n 8 logs/$1_tracrClusterLog.log > logs/clusterInfo
head -n 1 logs/clusterInfo; rm logs/clusterInfo
mv sequences/$1_predictedTracrRNAs.grouped.fasta.clstr clusters/

In [2]:
tRound, nClusters = 4, 0
clusteredResults = processClusterFile("clusters/All_%s_Predicted_TracrRNAs.grouped.fasta.clstr" % (gene))
seqs = fasta_index("sequences/All_%s_Predicted_TracrRNAs.fasta" % (gene),"fasta")
for cluster,clusterSeqIDs in clusteredResults.items():
    if len(clusterSeqIDs.members) == 1: continue 
    hasMultipleAsms = False
    baseID = list(clusterSeqIDs.members.keys())[0]
    baseID = baseID[:baseID.rfind("_")] #remove the solution #
    baseID = baseID[:baseID.rfind("_")] #remove the orf ID
    for seqID in clusterSeqIDs.members: hasMultipleAsms = (hasMultipleAsms or baseID not in seqID)
    if not hasMultipleAsms or len(clusterSeqIDs.members)<=1: continue
    with open("conseqs%i/%s.fasta" % (tRound,cluster.replace(" ","_")),"w") as fh:
        nClusters += 1
        for tracrSeqID in clusterSeqIDs.members: write(seqs[tracrSeqID],fh,"fasta")
print("There are %i clusters" %(nClusters))

There are 399 clusters


In [7]:
%%bash
bash LaunchStructureSearch.sh 3

Process is interrupted.


<h2><a id="round0Results">Round 0 - Results</a></h2>

In [ ]:
tRound = 0
infernalResults = ProcessInfernal(tRound,gene) # Results from the last structural search
generateColors(infernalResults,casRelatedProteins,tRound,gene)
clusters = infernalResults.structMapping.keys()
seqIDs = infernalResults.seqTracrs.keys()
dataMap = {}

In [ ]:
tRound = 3
infernalResults = ProcessInfernal(tRound,gene) # Results from the last structural search

In [ ]:
from easyFunctions import dump
clusterCounter,treeMap,revMap, missingIDs = {}, {}, {}, set()
for struct,ids in infernalResults.structMapping.items(): clusterCounter[struct] = len(ids)
for protID in casRelatedProteins:
    if protID not in infernalResults.seqTracrs: missingIDs.add(protID); continue
    biggestCluster =  findBiggestCluster(infernalResults.seqTracrs[protID].keys(),clusterCounter)
    treeMap[protID] = biggestCluster
    try: revMap[biggestCluster].add(protID)
    except: revMap[biggestCluster] = set([protID])
treeColors,colors = {},{}
for cluster, protIDs in revMap.items():
    colors[cluster]=color()
    for protID in protIDs: treeColors[protID] = colors[cluster]
dump(treeColors,"pickles/%s_StructureResultsTreeColors_Round%i.p" % (gene,tRound))
dump(colors,    "pickles/%s_StructureResultsColors_Round%i.p"     % (gene,tRound))
print("In round %i, %i assmebly had a predicted tracr (%.1f %%) with %i tracrRNA Structures" % (tRound,len(treeMap),len(treeMap)/float(len(casRelatedProteins))*100,len(revMap)))
print("\t%i still remaining" % (len(missingIDs)))

<h2><a id="round1Prep">Round 1 - Get the tracr results and re-run structural results from the hits</a></h2>

In [ ]:
tRound = 1
# casOperons = load(open("pickles/casOperonDataStructureW%s.p" % gene,"rb"))
clusterCMD = """
cd-hit-est -i sequences/tracrs/%s_Round%i_%s_TracrRNAs.fasta -o sequences/tracrs/%s_Round%i_%s_TracrRNAs.grouped.fasta -M 0 -d 0 -c .85 -s .9 -sc 1
"""
for clusterID, protIDs in revMap.items():
    with open('sequences/tracrs/%s_Round%i_%s_TracrRNAs.fasta' % (gene,tRound,clusterID),'w') as clusterFile:
        for protID in protIDs:
            tracr = infernalResults.seqTracrs[protID][clusterID]
            chrAsmName = "assemblies/pseudoChromos/%s.fasta" % (protID)
            seq = fasta_index(chrAsmName,"fasta")[protID]
            newSeq = seq.seq[tracr.location.start:tracr.location.end].upper()
            seq.seq = newSeq
            seq.description = ""
            write(seq,clusterFile,"fasta")
    seqs = fasta_index("sequences/tracrs/%s_Round%i_%s_TracrRNAs.fasta" % (gene,tRound,clusterID),"fasta")
    if len(seqs) == 1 :continue
    os.system(clusterCMD % (gene,tRound,clusterID,gene,tRound,clusterID))
    print("There are %i TRACRs in %s" % (len(seqs),clusterID))
    clusteredResults = processClusterFile("sequences/tracrs/%s_Round%i_%s_TracrRNAs.grouped.fasta.clstr" % (gene,tRound,clusterID))
    for cluster,clusterSeqIDs in clusteredResults.items():
        if len(clusterSeqIDs.members) == 1: continue 
        if len(clusterSeqIDs.members) == 2: 
            s1, s2 = list(clusterSeqIDs.members.keys())
            if clusterSeqIDs.members[s1] == 100.0 or clusterSeqIDs.members[s2] == 100.0: continue
        with open("conseqs%i/%s_%s.fasta" % (tRound,clusterID,cluster.replace("Cluster ","")),"w") as fh:
            for tracrSeqID in clusterSeqIDs.members: write(seqs[tracrSeqID],fh,"fasta")
    os.system("rm sequences/tracrs/%s_Round%i_%s_TracrRNAs.fasta sequences/tracrs/%s_Round%i_%s_TracrRNAs.grouped.fasta" % (gene,tRound,clusterID,gene,tRound,clusterID))

In [ ]:
%%bash
export fastaDir=conseqs2
for i in 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 # 26 * 20 is the max CPU limit per user
do
    #Initial Numbers
    squeue -u dooleys1 -o "%.8i %.9P %.18j %.8u %.2t %.10M %.6D %R" | grep Priority |wc -l > queuedJobs
    read njobs < queuedJobs
    ls $fastaDir/*.fasta 2>/dev/null | wc -l >remaining
    read whatsLeft < remaining

    #Check that something isn't still queued and that there is more to run
    while [ $njobs -gt 0 ] && [ $whatsLeft -gt 0 ]; do
        sleep 10
        squeue -u dooleys1 -o "%.8i %.9P %.18j %.8u %.2t %.10M %.6D %R" | grep Priority |wc -l > queuedJobs
        read njobs < queuedJobs
        ls $fastaDir/*.fasta 2>/dev/null | wc -l >remaining
        read whatsLeft < remaining
       # echo "2There are $whatsLeft fasta files remaining"
    done

    #Run the new job
    sbatch --job-name="StructureSearch$i" ../scripts/hpc/StructureSearch.sb
    sleep 10

    #Check how many jobs are running and that there is still work to be done
    squeue -u dooleys1 -o "%.8i %.9P %.18j %.8u %.2t %.10M %.6D %R" | wc -l > totalJobs
    read tjobs < totalJobs
    ls $fastaDir/*.fasta 2>/dev/null |wc -l >remaining
    read whatsLeft < remaining
    if [ $tjobs -ge 26 -o $whatsLeft -eq 0 ]; then
        break
    fi
done
rm queuedJobs totalJobs remaining

<h2><a id="round1Results">Round 1 - Results</a></h2>

In [ ]:
tRound = 1
infernalResults0 = ProcessInfernal(0,gene)
infernalResults1 = ProcessInfernal(tRound,gene)
hasResultInBoth = set(infernalResults0.seqTracrs.keys()).intersection(infernalResults1.seqTracrs.keys())
len(hasResultInBoth)

infernalResults = infernalResults1

## Round 2 Iterative Method

In [ ]:
tRound = 2
from easyFunctions import dump
clusterCounter,treeMap,revMap, missingIDs = {}, {}, {}, set()
for struct,ids in infernalResults.structMapping.items(): clusterCounter[struct] = len(ids)
for protID in casRelatedProteins:
    if protID not in infernalResults.seqTracrs: missingIDs.add(protID); continue
    biggestCluster =  findBiggestCluster(infernalResults.seqTracrs[protID].keys(),clusterCounter)
    treeMap[protID] = biggestCluster
    try: revMap[biggestCluster].add(protID)
    except: revMap[biggestCluster] = set([protID])
        

# casOperons = load(open("pickles/casOperonDataStructureW%s.p" % gene,"rb"))
clusterCMD = """
cd-hit-est -i sequences/tracrs/%s_Round%i_%s_TracrRNAs.fasta -o sequences/tracrs/%s_Round%i_%s_TracrRNAs.grouped.fasta -M 0 -d 0 -c .85 -s .9 -sc 1
"""
for clusterID, protIDs in revMap.items():
    with open('sequences/tracrs/%s_Round%i_%s_TracrRNAs.fasta' % (gene,tRound,clusterID),'w') as clusterFile:
        for protID in protIDs:
            tracr = infernalResults.seqTracrs[protID][clusterID]
            chrAsmName = "assemblies/pseudoChromos/%s.fasta" % (protID)
            seq = fasta_index(chrAsmName,"fasta")[protID]
            newSeq = seq.seq[tracr.location.start:tracr.location.end].upper()
            seq.seq = newSeq
            seq.description = ""
            write(seq,clusterFile,"fasta")
    seqs = fasta_index("sequences/tracrs/%s_Round%i_%s_TracrRNAs.fasta" % (gene,tRound,clusterID),"fasta")
    if len(seqs) == 1 :continue
    os.system(clusterCMD % (gene,tRound,clusterID,gene,tRound,clusterID))
    print("There are %i TRACRs in %s" % (len(seqs),clusterID))
    clusteredResults = processClusterFile("sequences/tracrs/%s_Round%i_%s_TracrRNAs.grouped.fasta.clstr" % (gene,tRound,clusterID))
    for cluster,clusterSeqIDs in clusteredResults.items():
        if len(clusterSeqIDs.members) == 1: continue 
        if len(clusterSeqIDs.members) == 2: 
            s1, s2 = list(clusterSeqIDs.members.keys())
            if clusterSeqIDs.members[s1] == 100.0 or clusterSeqIDs.members[s2] == 100.0: continue
        with open("conseqs%i/%s_%s.fasta" % (tRound,clusterID,cluster.replace("Cluster ","")),"w") as fh:
            for tracrSeqID in clusterSeqIDs.members: write(seqs[tracrSeqID],fh,"fasta")
    os.system("rm sequences/tracrs/%s_Round%i_%s_TracrRNAs.fasta sequences/tracrs/%s_Round%i_%s_TracrRNAs.grouped.fasta" % (gene,tRound,clusterID,gene,tRound,clusterID))
# treeColors,colors = {},{}
# for cluster, protIDs in revMap.items():
#     colors[cluster]=color()
#     for protID in protIDs: treeColors[protID] = colors[cluster]
# dump(treeColors,"pickles/%s_StructureResultsTreeColors_Round%i.p" % (gene,tRound))
# dump(colors,    "pickles/%s_StructureResultsColors_Round%i.p"     % (gene,tRound))
# print("In round %i, %i assmebly had a predicted tracr (%.1f %%) with %i tracrRNA Structures" % (tRound,len(treeMap),len(treeMap)/float(len(casRelatedAssemblies))*100,len(revMap)))
# print("\t%i still remaining" % (len(missingIDs)))

In [ ]:
def findBiggestCluster(clusters, clusterMap): return Counter(dict((k, clusterMap[k]) for k in (clusters))).most_common(1)[0][0] 
def getNonOverlapping(topTracr,allTracrs):
    tracrCoords = set(range(topTracr.location.start,topTracr.location.end+1))
    keepers = set([topTracr])
    for tracr in allTracrs:
        if tracr.location.start not in tracrCoords and tracr.location.end not in tracrCoords:
            tracrCoords = tracrCoords.union(set(range(tracr.location.start,tracr.location.end+1)))
            keepers.add(tracr)
    
#     tracrCoords = list(tracrCoords)
#     tracrCoords.sort()
#     prev  = tracrCoords[0]
#     print(prev, end=" ")
#     for val in tracrCoords[1:]:
#         if val-prev != 1:
#             print(val)
#         prev=val
    for tr in keepers:
        print(tr)
        
    return keepers
    
    

In [ ]:
def combineTracrs(inf1,inf2,top1,top2,protID,operon,seqs,clusters):
    topClus1 = findBiggestCluster(inf1.seqTracrs[protID].keys(),top1)
    topTracr = inf1.seqTracrs[protID][topClus1]
    antiCoords = set()
    
    tracrs = set()
    for coord in operon.crispr.antiRepeats: antiCoords = antiCoords.union(set(range(coord.start,coor.end+1)))
    notMatching        
    
    
    
    topClus2 = findBiggestCluster(inf2.seqTracrs[protID].keys(),top2)
    print(protID,inf1.seqTracrs[protID][topClus1],inf2.seqTracrs[protID][topClus2])
    print("\tAnti-Repeats:")
    for coord in operon.crispr.antiRepeats:
        print("\t\t",coord)
    

    tracrs1 = getNonOverlapping(inf1.seqTracrs[protID][topClus1],inf1.seqTracrs[protID].values())
    tracrs2 = getNonOverlapping(inf2.seqTracrs[protID][topClus2],inf2.seqTracrs[protID].values())
    print("\n")
    
    

    
    
    
#     print(tracrLocs[-1])
#     print("%s T1 %i" % (protID,len(tracrs1)))
#     for cluster, tracr in tracrs1.items(): 
# #         for t2 in tracrLocs:
            
#         print('\t',tracr)
#         tracrLocs.add(tracr)
#     print("%s T2 %i" % (protID,len(tracrs2)))
#     for cluster, tracr in tracrs2.items(): 
#         print('\t',tracr)
#         tracrLocs.add(tracr)
#     print(len(tracrLocs))

In [ ]:
inf0Counter = Counter()
for clustID, ids in infernalResults0.structMapping.items(): inf0Counter[clustID] = len(ids)
inf1Counter = Counter()
for clustID, ids in infernalResults1.structMapping.items(): inf1Counter[clustID] = len(ids)

In [ ]:
operon.crispr.antiRepeats

In [ ]:
combinedResultsBySeq,combinedResultsByLocation = {},{} 
for i, protID in enumerate(hasResultInBoth):
    operon = casOperons[protID]
    tracrLocs = combineTracrs(infernalResults0, infernalResults1, inf0Counter, inf1Counter, protID,operon,combinedResultsBySeq,combinedResultsByLocation)
    if i == 5: break

<h2><a id="round2Prep">Round 2 - Take the results from the first 2 runs, combine the unique runs and run again</a></h2>

In [ ]:
def filterTracrs(tracrs1,tracrs2,consRepeat,asmSeq):
    #TODO: check the output of infernal and make sure we aren't stomping on multiple tracrs in same cluster out
    #What does the union of tracrs look like?
    allTracrs = set()
    for cluster, tracr in tracrs1.items():
        try:
            tracrSeq = tracr.seq(asmSeq)
            tracrIdent = duplexPercIdent(tracrSeq,consRepeat)
            if tracrIdent >= 35.0: allTracrs.add(tracr)
        except: print(consRepeat,asmSeq)
    for cluster,tracr in tracrs2.items():
        tracrSeq = tracr.seq(asmSeq)
        tracrIdent = duplexPercIdent(tracrSeq,consRepeat)
        if tracrIdent >= 35.0: allTracrs.add(tracr)
    return allTracrs
def duplexPercIdent(tracrRNA,crRNA):
    crLen = len(crRNA)
    if len(tracrRNA) <= crLen: return 0.0
    fmatches,rmatches = 0,0
    for i in range(crLen): 
        fmatches += int(tracrRNA[i]==crRNA[i])
        rmatches += int(tracrRNA[-(i+1)]==crRNA[-(i+1)])
    revComp = RC(crRNA)
    rfmatches,rrmatches = 0,0
    for i in range(crLen): 
        rfmatches += int(tracrRNA[i]==revComp[i])
        rrmatches += int(tracrRNA[-(i+1)]==revComp[-(i+1)])
    crLen = float(crLen)
    return max(fmatches/crLen, rmatches/crLen, rfmatches/crLen, rrmatches/crLen)*100

tRound = 2
possibleSol = open("sequences/%s_predictedTracrRNAs_Round%i.fasta" % (gene,tRound),"w")
# casOperons = load(open("pickles/casOperonDataStructureW%s.p" % gene,"rb"))
counter = 0
tracrCount = 0
for protID in casRelatedAssemblies:
    if protID in infernalResults0.seqTracrs and protID in infernalResults1.seqTracrs:
        counter+=1
        operon = casOperons[protID]
        if operon.seq is None:
            chrAsmName = "assemblies/pseudoChromos/%s.fasta" % (protID)
            operon.seq = fasta_index(chrAsmName,'fasta')[protID]
        validTracrs = filterTracrs(infernalResults0.seqTracrs[protID],infernalResults1.seqTracrs[protID],list(operon.crispr.consensusRepeats)[0],operon.seq)
        for tracr in validTracrs:
            tracrCount += 1
            possibleSol.write((">Seq_%i\n%s\n")% (tracrCount,tracr.seq(operon.seq.seq)))
possibleSol.close()  
tracrCount,counter

In [ ]:
tRound = 2
tracrCount = 0
possibleSol = open("sequences/%s_predictedTracrRNAs_Round%i.fasta" % (gene,tRound),"w")
for protID in casRelatedAssemblies:
    if protID in infernalResults0.seqTracrs and protID in infernalResults1.seqTracrs:
        operon = casOperons[protID]
        if operon.seq is None:
            chrAsmName = "assemblies/pseudoChromos/%s.fasta" % (protID)
            operon.seq = fasta_index(chrAsmName,'fasta')[protID]
        for id, tracr in infernalResults0.seqTracrs[protID].items():
            tracrCount += 1
            possibleSol.write((">Seq_%i\n%s\n")% (tracrCount,tracr.seq(str(operon.seq.seq))))
        for id, tracr in infernalResults1.seqTracrs[protID].items():
            tracrCount += 1
            possibleSol.write((">Seq_%i\n%s\n")% (tracrCount,tracr.seq(str(operon.seq.seq))))
possibleSol.close()  
tracrCount,counter

In [ ]:
%%bash -s $gene $tRound
cd-hit-est -i sequences/$1_predictedTracrRNAs_Round$2.fasta -o sequences/$1_predictedTracrRNAs_Round$2.grouped.fasta -M 0 -d 0 -c .85 -s .9 -sc 1 >logs/$1_tracrClusterLog.log
tail -n 8 logs/$1_tracrClusterLog.log > logs/clusterInfo
head -n 1 logs/clusterInfo; rm logs/clusterInfo
mv sequences/$1_predictedTracrRNAs_Round$2.grouped.fasta.clstr clusters/

In [ ]:
tRound = 2
numClusts = 0 
clusteredResults = processClusterFile("clusters/%s_predictedTracrRNAs_Round%i.grouped.fasta.clstr" % (gene,tRound))
seqs = fasta_index("sequences/%s_predictedTracrRNAs_Round%i.fasta" % (gene,tRound),"fasta")
for cluster,clusterSeqIDs in clusteredResults.items():
    if len(clusterSeqIDs.members) == 1: continue 
#     if len(clusterSeqIDs.members) == 2:
    with open("conseqs%i/%s.fasta" % (tRound,cluster.replace(" ","_")),"w") as fh:
        numClusts+=1
        for tracrSeqID in clusterSeqIDs.members: write(seqs[tracrSeqID],fh,"fasta")
numClusts

In [ ]:
%%bash
bash LaunchStructureSearch.sh

<h2><a id="round2Results">Round 2 - Results</a></h2>

In [ ]:
tRound = 2
infernalResults = ProcessInfernal(tRound,gene)

<h2><a id="combinedResults">Results from all 3 rounds</a></h2>

In [ ]:
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq

combinedResults = set()
infernalResults = [0,1,2,3,4]
# allResults =  open("sequences/%s_combinedStructuralResults.fasta" % (gene),"w")
for i in range(0,3):
    beforeNum = len(combinedResults)
    infernalResults[i] = ProcessInfernal(i,gene)
    combinedResults = combinedResults.union(infernalResults[i].seqTracrs.keys())
    print("Round %i found %i more results for a total of %i = %.2f %%" % 
          (i, len(combinedResults)-beforeNum,len(combinedResults),
          len(combinedResults)/float(len(casRelatedProteins))*100))
#     for seqID,clusterIDs in infernalResults[i].seqTracrs.items():
#         nuclSeq = casOperons[seqID].seq
#         for clusterID in clusterIDs:
#             tracr = infernalResults[i].seqTracrs[seqID][clusterID]
#             rec = SeqRecord(nuclSeq.seq[tracr.location.start:tracr.location.end], id=seqID+"___"+clusterID, name="", description='')
#             write(rec,allResults,"fasta")
# allResults.close()    

In [ ]:
def generateColors(infernalResults,casAssemblies,tRound,gene):
    clusterCounter, treeColors, colors, usedIDs = Counter(), {}, {}, set()          #,treeMap,revMap, missingIDs = {}, {}, {}, set()
    for struct,ids in infernalResults.structMapping.items(): clusterCounter[struct] = len(ids)
    
    for clusterID in clusterCounter.most_common(len(infernalResults.structMapping)):
        idsWithNoColor = infernalResults.structMapping[clusterID[0]].difference(usedIDs)
        if len(idsWithNoColor) <= 1 : continue
        colors[clusterID]=genColor()
        for protID in idsWithNoColor: treeColors[protID] = colors[clusterID]
        usedIDs = usedIDs.union(infernalResults.structMapping[clusterID[0]])
    dump(treeColors,open("pickles/%s_StructureResultsTreeColors_Round%i.p" % (gene,tRound),'wb'))
    dump(colors,    open("pickles/%s_StructureResultsColors_Round%i.p"     % (gene,tRound),'wb'))
    print("In round %i, %i assmebly had a predicted tracr (%.1f %%) with %i tracrRNA Structures" % 
         (tRound,len(usedIDs),len(usedIDs)/float(len(casAssemblies))*100,len(colors)))
    print("\t%i still remaining" % (len(set(casAssemblies.keys()).difference(usedIDs))))     

In [ ]:
#from InfernalResults import generateColors
tRound = 1
#infernalResults = ProcessInfernal(tRound,gene) # Results from the last structural search
generateColors(infernalResults,casRelatedAssemblies,tRound,gene)

In [ ]:
missingIDs = set(casRelatedAssemblies.keys()).difference(combinedResults)
clusteredResults = processClusterFile("clusters/%s-Like-clustered.faa.clstr" % (gene))
counter = 0
casIDs = set(casRelatedAssemblies.keys())
for protID in missingIDs:
    cluster= clusteredResults.revMap[protID]
    if len(clusteredResults[cluster].members)==1: continue
    longestCRISPR = ''
    maxRepeats = 0
    for member in clusteredResults[cluster].members:
        operon = casOperons[member].crispr[member]
        crType = 'pilerCR'
        if crType not in operon: crType = 'minCED'
        locus = operon[crType]
        if len(locus.repeatCoords) > maxRepeats:
            longestCRISPR = member
            maxRepeats = len(locus.repeatCoords)
    counter +=1
    casIDs.remove(protID)
    casIDs.add(longestCRISPR)
counter

In [ ]:
corrected = {}
for protID in casIDs: corrected[protID[:protID.rfind("_")]] = protID 
casIDs = corrected

In [ ]:
newCasRepFile = open("assemblies/Cas9_Representative_Assemblies_2.fasta","w")
allNuclSeqs = {}
counter = 0
for rec in parse("assemblies/All_Cas9_Representative_Assemblies.fasta","fasta"):
    if rec.id in casIDs:
        rec.id = casIDs[rec.id]
        chrAsmName = "assemblies/pseudoChromos/%s.fasta" % (rec.id)
        if not path.exists(chrAsmName):
            with open(chrAsmName,'w') as fh: write(rec,fh,"fasta")
        write(rec,newCasRepFile,'fasta')
        try: length = len(casOperons[rec.id].seq)
        except: 
            counter+=1
            locus = casOperons[rec.id]
            setattr(locus,'seq',rec)
            locus.seq = rec
dump(casOperons, "pickles/%s_Operons.p" % gene)       
newCasRepFile.close()  
counter,len(casIDs)

In [ ]:
%%bash
head assemblies/Cas9_Representative_Assemblies_2.fasta

In [ ]:
tRound = 2
# casOperons = load(open("pickles/casOperonDataStructureW%s.p" % gene,"rb"))
clusterCMD = """
cd-hit-est -i sequences/tracrs/%s_Round%i_%s_TracrRNAs.fasta -o sequences/tracrs/%s_Round%i_%s_TracrRNAs.grouped.fasta -M 0 -d 0 -c .90 -s .9 -sc 1
"""
for clusterID, protIDs in revMap.items():
    with open('sequences/tracrs/%s_Round%i_%s_TracrRNAs.fasta' % (gene,tRound,clusterID),'w') as clusterFile:
        for protID in protIDs:
            baseID = protID[:protID.rfind("_")]
            tracr = infernalResults.seqTracrs[baseID][clusterID]
            chrAsmName = "assemblies/pseudoChromos/%s.fasta" % (protID)
            seq = fasta_index(chrAsmName,"fasta")[protID]
            if tracr.location.start > tracr.location.end: 
                temp = tracr.location.start
                tracr.location.start = tracr.location.end
                tracr.location.end = temp
            newSeq = seq.seq[tracr.location.start:tracr.location.end].upper()
            seq.seq = newSeq
            seq.description = ""
            write(seq,clusterFile,"fasta")
    seqs = fasta_index("sequences/tracrs/%s_Round%i_%s_TracrRNAs.fasta" % (gene,tRound,clusterID),"fasta")
    if len(seqs) == 1 :continue
    os.system(clusterCMD % (gene,tRound,clusterID,gene,tRound,clusterID))
    print("There are %i TRACRs in %s" % (len(seqs),clusterID))
    clusteredResults = processClusterFile("sequences/tracrs/%s_Round%i_%s_TracrRNAs.grouped.fasta.clstr" % (gene,tRound,clusterID))
    for cluster,clusterSeqIDs in clusteredResults.items():
        if len(clusterSeqIDs.members) == 1: continue 
        if len(clusterSeqIDs.members) == 2: 
            s1, s2 = list(clusterSeqIDs.members.keys())
            if clusterSeqIDs.members[s1] == 100.0 or clusterSeqIDs.members[s2] == 100.0: continue
        with open("conseqs%i/%s_%s.fasta" % (tRound,clusterID,cluster.replace("Cluster ","")),"w") as fh:
            for tracrSeqID in clusterSeqIDs.members:
                write(seqs[tracrSeqID],fh,"fasta")
    os.system("rm sequences/tracrs/%s_Round%i_%s_TracrRNAs.fasta sequences/tracrs/%s_Round%i_%s_TracrRNAs.grouped.fasta" % (gene,tRound,clusterID,gene,tRound,clusterID))

In [ ]:
allAssemblies = fasta_index("assemblies/All_%s_Representative_Assemblies.fasta" % (gene),'fasta')
len(allAssemblies)

In [ ]:
breakPoints = set()
for i in range(0,len(allAssemblies),500): breakPoints.add(i)
breakPoints.remove(0)

<h2><a id='everything'>Doing EVERYTHING at the same time!</a></h2>

[Home](#libraries)


In [ ]:
#     locus.annotate(casRelatedAssemblies[protID], chrAsmName)
#     if locus.hasNRegion: 
#         nCounter += 1
#         hasNRegionSet.add(protID)
#     allLoci[protID] = locus
    
#     #Step 2c
#     tmpFasta = open("tmp/possibleTracrs.fasta","w")
    
    #Step 2d
    chrRec = fasta_index(chrAsmName,"fasta")
    chrSequence = str(chrRec[list(chrRec.keys())[0]].seq)
#     terminusSeqs, index = locus.getTerminusRepeats(chrSequence,index)
#     tmpFasta.write(terminusSeqs)
    
#     #Collect some stats
# #     startCoord = locus.repeatCoords[0]
# #     endCoord = locus.repeatCoords[-1]
# #     seqlen = len(casRelatedAssemblies[protID])
# #     minDist = min(seqlen-startCoord.start,seqlen-startCoord.end,seqlen-endCoord.start,seqlen-endCoord.end)
# #     minDist =  min(minDist,startCoord.start,startCoord.end,endCoord.start,endCoord.end)

#     ##Step 2e - Write sequence surounding hits to the fasta file
#     for rec in boundaryHits: index = getSurroundingSequence(tmpFasta,index,chrSequence,rec,500)
    
#     ##Step 2f - Look for terminators in the possible tracrRNAs
#     tmpFasta.close()
#     cmd = "~/bin/Arnold/erpin ~/bin/Arnold/rho-indep.epn tmp/possibleTracrs.fasta -1,4 -add 1 4 2 -pcw 1 -cutoff 100% >tmp/rhoInd.out"
#     system(cmd)
    
#     ##Setp 2j - process the terminators that were found
#     erpOut = ErpinOut()
#     if len(erpOut.terminators)==0: 
#         print ("\nThis ref has nothing: %s\n" % protID)
#         noPredictedTracr.append(protID)
#         if locus.hasNRegion: n_and_no_possibles +=1
#     else: newSolutions[protID] = erpOut

# dump(newSolutions, open("pickles/%s_predictedTracrRNAs.p" % (gene), "wb"))
numPos, numTerminators, tracrSeqDict = {}, {}, {}
newSolutions = load(open("pickles/%s_predictedTracrRNAs.p" % (gene),"rb"))
index = 0
sizeDist = []
ignoreSol = 0
possibleSol = open("sequences/%s_predictedTracrRNAs.fasta" % (gene),"w")
for ref,psol in newSolutions.items():
    for terminator in psol.terminators:
        seq = psol.records[terminator.name]
        solSize = 0
        #                               ANTI-REPEAT############################A
            #############################RC-ANTI-REPEAT
        if terminator.upstream and not terminator.strand: tracrSeq = seq[terminator.Rholocation.start-1:].upper()
        elif not terminator.upstream and terminator.strand: tracrSeq = seq[:terminator.Rholocation.end].upper()
        else: continue #We may have found a termination signal but it is on the wrong strand to code the anti-repeat
        solSize = len(tracrSeq)
        sizeDist.append(solSize)
        if solSize>350 or tracrSeq.count("N")>=4:
            ignoreSol+=1
            continue
        index += 1
        tracrSeqDict["%s_%i" % (ref,index)] = tracrSeq
        possibleSol.write(">%s_%i\n%s\n" % (ref,index,tracrSeq))

possibleSol.close()
print ("Ignored %i solutions" % ignoreSol)
print ("Done", index, "possible solutions")


In [ ]:
casOperons = load(open("pickles/casOperonDataStructureW%s.p" % gene,"rb"))
moreThan1Array = 0
nminced,multicounter = 0,0
for asmBase,asmOperon in casOperons.casOperon.items():
#     for key in asmOperon.crisprs:
        
    if len(asmOperon.crisprs) > 1: multicounter += 1
    crisprs = asmOperon.crisprs[list(asmOperon.crisprs.keys())[0]]
    
    if 'pilerCR' not in crisprs: 
        print(asmBase)
        print(crisprs)
        break
        nminced+=1; continue
    
    moreThan1Array += int(len(crisprs['pilerCR'].consensusRepeat)>1)
crlocus = casOperons.casOperon['GCF_000024805.1'].crisprs['NC_013440.1_ORF5729']['pilerCR']
moreThan1Array,nminced,multicounter

In [ ]:
allClusters, allClusterSeqIDs = {},{}
for line in open("clusters/%s_predictedTracrRNAs.grouped.fasta.clstr" % (gene)):
    if ">Cluster" in line: clusterID = line.strip().replace(">","")
    else:
        seqID = line[line.find(">")+1:line.find("...")] 
        seqID = seqID[:seqID.rfind("_")]
        try:allClusters[clusterID].add(seqID)
        except:allClusters[clusterID]=set([seqID])
        try:allClusterSeqIDs[seqID].add(clusterID)
        except:allClusterSeqIDs[seqID]=set([clusterID])
xs,ys,zs=[],[],[]
remove=set()
colors,TreeColors={},{}
clusterDist=[]
for cluster, ids in allClusters.items():
    clusterDist.append(len(ids))
    if len(ids) < 2: remove.add(cluster)
    else:
        colors[cluster]=color()
        for id in ids:TreeColors[id.replace(".","_")] = colors[cluster]
for id in remove: del allClusters[id]

print ("\tTotal number of clusters:",len(allClusters))
print ("\tNumber of nodes covered:",len(TreeColors))
# fig = plt.figure()
# ax = fig.add_subplot(111, projection='3d')
# xs.append(len(allClusters))
# ys.append(len(TreeColors))
# zs.append(i)
# ax.scatter(ys,xs,zs)    
# # xs.reverse()
# # ys.reverse()
# # plt.plot(xs, ys, 'ro')
# plt.show()

In [ ]:
%%bash -s $gene
cd-hit-est -i sequences/$1_predictedTracrRNAs.fasta -o sequences/$1_predictedTracrRNAs.grouped.fasta -M 0 -d 0 -c .90 -T 0 >logs/$1_tracrClusterLog.log
tail -n 8 logs/$1_tracrClusterLog.log > logs/clusterInfo
head -n 1 logs/clusterInfo; rm logs/clusterInfo
mv sequences/$1_predictedTracrRNAs.grouped.fasta.clstr clusters/

In [ ]:
allClusters, allClusterSeqIDs = {},{}
for line in open("clusters/%s_predictedTracrRNAs.grouped.fasta.clstr" % (gene)):
    if ">Cluster" in line: clusterID = line.strip().replace(">","")
    else:
        seqID = line[line.find(">")+1:line.find("...")] 
        seqID = seqID[:seqID.rfind("_")]
        try:allClusters[clusterID].add(seqID)
        except:allClusters[clusterID]=set([seqID])
        try:allClusterSeqIDs[seqID].add(clusterID)
        except:allClusterSeqIDs[seqID]=set([clusterID])
xs,ys,zs=[],[],[]
remove=set()
colors,TreeColors={},{}
clusterDist=[]
for cluster, ids in allClusters.items():
    clusterDist.append(len(ids))
    if len(ids) < 2: remove.add(cluster)
    else:
        colors[cluster]=color()
        for id in ids:TreeColors[id.replace(".","_")] = colors[cluster]
for id in remove: del allClusters[id]

print ("\tTotal number of clusters:",len(allClusters))
print ("\tNumber of nodes covered:",len(TreeColors))

In [ ]:
tmpDict = {}
tracrSeqDict = fasta_index("sequences/%s_predictedTracrRNAs.fasta" % (gene),"fasta")
for id,seq in tracrSeqDict.items(): tmpDict[id[:id.rfind("_")]]=str(seq.seq).upper()
tracrSeqDict = tmpDict  
for cluster, seqIDList in allClusters.items():
    clusterFileName = "conseqs0/"+cluster.replace(" ","_")+".fasta"
    fh = open(clusterFileName,"w")
    clusterStats = []
    for seqID in seqIDList: clusterStats.append(len(tracrSeqDict[seqID]))
    clusterStats = Series(clusterStats)
    keepSeqs = clusterStats[~((clusterStats-clusterStats.mean()).abs() > clusterStats.std())]
    minSeq, maxSeq = keepSeqs.min(), keepSeqs.max()
    index=0
    for seqID in seqIDList: 
        seqLen = len(tracrSeqDict[seqID])
        if seqLen >= minSeq and seqLen <= maxSeq:
            index+=1
            fh.write(">%s\n%s\n" % (seqID,tracrSeqDict[seqID]))
    fh.close()
    if index <= 1: system("rm %s" % (clusterFileName)); continue
    
    os.system("cd-hit-est -i %s -o %s_cluster -M 0 -d 0 -c .90 -T 0 -sc 1 >/dev/null" % (clusterFileName,clusterFileName))
    clusteredResults = processClusterFile("%s_cluster.clstr" % (clusterFileName))
    fh = open(clusterFileName,"w")
    index = 0 
    for seqID,direction in clusteredResults.filter():
        index +=1
        if direction: fh.write(">%s\n%s\n" % (seqID,str(tracrSeqDict[seqID])))
        else: fh.write(">%s\n%s\n" % (seqID, reverse_complement(tracrSeqDict[seqID])))
    fh.close()
    if index <= 1: system("rm %s" % (clusterFileName))
#     else: print( clusterFileName,"(",index,"/",len(seqIDList), ")[%i,%i]" % (minSeq,maxSeq),clusterStats.std() )
    system("rm %s_cluster*" % (clusterFileName))

### Read the Structure Search Results

In [ ]:
tRound = 0
infernalResults = ProcessInfernal(tRound,gene)

In [ ]:
tRound = 1
infernalResults = ProcessInfernal(tRound,gene)

### Look for structures overlapping the same sequence to combine the sequences into the same structures

In [ ]:
infernalResults = ProcessInfernal(tRound,gene)

clusterCounter,treeMap,revMap, missingIDs = {}, {}, {}, set()
for struct,ids in infernalResults.structMapping.items(): clusterCounter[struct] = len(ids)
missingCounter = 0
for protID in casRelatedAssemblies:
    baseID = protID[:protID.rfind("_")]
    if baseID not in infernalResults.seqTracrs: 
        missingCounter += 1; 
        missingIDs.add(protID)
        continue
    biggestCluster =  findBiggestCluster(infernalResults.seqTracrs[baseID].keys(),clusterCounter)
    treeMap[protID] = biggestCluster
    try: revMap[biggestCluster].add(protID)
    except: revMap[biggestCluster] = set([protID])
missingCounter, len(revMap)
treeColors,colors = {},{}
for cluster, protIDs in revMap.items():
    colors[cluster]=color()
    for protID in protIDs: treeColors[protID] = colors[cluster]
dump(treeColors,open("pickles/%s_StructureResultsTreeColors_Round%i.p" % (gene,tRound),"wb"))
dump(colors,    open("pickles/%s_StructureResultsColors_Round%i.p"     % (gene,tRound),"wb"))

# Round 2

In [ ]:
tRound = 2
# casOperons = load(open("pickles/casOperonDataStructureW%s.p" % gene,"rb"))
clusterCMD = """
cd-hit-est -i sequences/tracrs/%s_Round%i_%s_TracrRNAs.fasta -o sequences/tracrs/%s_Round%i_%s_TracrRNAs.grouped.fasta -M 0 -d 0 -c .90 -s .9 -sc 1
"""
for clusterID, protIDs in revMap.items():
    with open('sequences/tracrs/%s_Round%i_%s_TracrRNAs.fasta' % (gene,tRound,clusterID),'w') as clusterFile:
        for protID in protIDs:
            baseID = protID[:protID.rfind("_")]
            tracr = infernalResults.seqTracrs[baseID][clusterID]
            chrAsmName = "assemblies/pseudoChromos/%s.fasta" % (protID)
            seq = fasta_index(chrAsmName,"fasta")[protID]
            if tracr.location.start > tracr.location.end: 
                temp = tracr.location.start
                tracr.location.start = tracr.location.end
                tracr.location.end = temp
            newSeq = seq.seq[tracr.location.start:tracr.location.end].upper()
            seq.seq = newSeq
            seq.description = ""
            write(seq,clusterFile,"fasta")
    seqs = fasta_index("sequences/tracrs/%s_Round%i_%s_TracrRNAs.fasta" % (gene,tRound,clusterID),"fasta")
    if len(seqs) == 1 :continue
    os.system(clusterCMD % (gene,tRound,clusterID,gene,tRound,clusterID))
    print("There are %i TRACRs in %s" % (len(seqs),clusterID))
    clusteredResults = processClusterFile("sequences/tracrs/%s_Round%i_%s_TracrRNAs.grouped.fasta.clstr" % (gene,tRound,clusterID))
    for cluster,clusterSeqIDs in clusteredResults.items():
        if len(clusterSeqIDs.members) == 1: continue 
        if len(clusterSeqIDs.members) == 2: 
            s1, s2 = list(clusterSeqIDs.members.keys())
            if clusterSeqIDs.members[s1] == 100.0 or clusterSeqIDs.members[s2] == 100.0: continue
        with open("conseqs%i/%s_%s.fasta" % (tRound,clusterID,cluster.replace("Cluster ","")),"w") as fh:
            for tracrSeqID in clusterSeqIDs.members:
                write(seqs[tracrSeqID],fh,"fasta")
    os.system("rm sequences/tracrs/%s_Round%i_%s_TracrRNAs.fasta sequences/tracrs/%s_Round%i_%s_TracrRNAs.grouped.fasta" % (gene,tRound,clusterID,gene,tRound,clusterID))
    
    

In [ ]:
tRound = 2
infernalResults = ProcessInfernal(tRound,gene)

In [ ]:
tRound = 3
# casOperons = load(open("pickles/casOperonDataStructureW%s.p" % gene,"rb"))
clusterCMD = """
cd-hit-est -i sequences/tracrs/%s_Round%i_%s_TracrRNAs.fasta -o sequences/tracrs/%s_Round%i_%s_TracrRNAs.grouped.fasta -M 0 -d 0 -c .85 -s .9 -sc 1
"""
for clusterID, protIDs in revMap.items():
    with open('sequences/tracrs/%s_Round%i_%s_TracrRNAs.fasta' % (gene,tRound,clusterID),'w') as clusterFile:
        for protID in protIDs:
            baseID = protID[:protID.rfind("_")]
            tracr = infernalResults.seqTracrs[baseID][clusterID]
            chrAsmName = "assemblies/pseudoChromos/%s.fasta" % (protID)
            seq = fasta_index(chrAsmName,"fasta")[protID]
            if tracr.location.start > tracr.location.end: 
                temp = tracr.location.start
                tracr.location.start = tracr.location.end
                tracr.location.end = temp
            newSeq = seq.seq[tracr.location.start:tracr.location.end].upper()
            seq.seq = newSeq
            seq.description = ""
            write(seq,clusterFile,"fasta")
    seqs = fasta_index("sequences/tracrs/%s_Round%i_%s_TracrRNAs.fasta" % (gene,tRound,clusterID),"fasta")
    if len(seqs) == 1 :continue
    os.system(clusterCMD % (gene,tRound,clusterID,gene,tRound,clusterID))
    print("There are %i TRACRs in %s" % (len(seqs),clusterID))
    clusteredResults = processClusterFile("sequences/tracrs/%s_Round%i_%s_TracrRNAs.grouped.fasta.clstr" % (gene,tRound,clusterID))
    for cluster,clusterSeqIDs in clusteredResults.items():
        if len(clusterSeqIDs.members) == 1: continue 
        if len(clusterSeqIDs.members) == 2: 
            s1, s2 = list(clusterSeqIDs.members.keys())
            if clusterSeqIDs.members[s1] == 100.0 or clusterSeqIDs.members[s2] == 100.0: continue
        with open("conseqs%i/%s_%s.fasta" % (tRound,clusterID,cluster.replace("Cluster ","")),"w") as fh:
            for tracrSeqID in clusterSeqIDs.members:
                write(seqs[tracrSeqID],fh,"fasta")
    os.system("rm sequences/tracrs/%s_Round%i_%s_TracrRNAs.fasta sequences/tracrs/%s_Round%i_%s_TracrRNAs.grouped.fasta" % (gene,tRound,clusterID,gene,tRound,clusterID))

In [ ]:
def findBiggestCluster(clusters, clusterMap): return Counter(dict((k, clusterMap[k]) for k in (clusters))).most_common(1)[0][0] 
clusterCounter,treeMap,revMap, missingIDs = {}, {}, {}, set()
for struct,ids in infernalResults.structMapping.items(): clusterCounter[struct] = len(ids)
missingCounter = 0
for protID in casRelatedAssemblies:
    baseID = protID[:protID.rfind("_")]
    if baseID not in infernalResults.seqTracrs: 
        missingCounter += 1; 
        missingIDs.add(protID)
        continue
    biggestCluster =  findBiggestCluster(infernalResults.seqTracrs[baseID].keys(),clusterCounter)
    treeMap[protID] = biggestCluster
    try: revMap[biggestCluster].add(protID)
    except: revMap[biggestCluster] = set([protID])
missingCounter, len(revMap)
treeColors,colors = {},{}
for cluster, protIDs in revMap.items():
    colors[cluster]=color()
    for protID in protIDs: treeColors[protID] = colors[cluster]
dump(treeColors,open("pickles/%s_StructureResultsTreeColors_Round%i.p" % (gene,tRound),"wb"))
dump(colors,    open("pickles/%s_StructureResultsColors_Round%i.p"     % (gene,tRound),"wb"))

In [ ]:
newSolutions = load(open("pickles/%s_predictedTracrRNAs.p" % (gene), "rb"))
hasPossibles = 0
for protID in missingIDs: hasPossibles += int(protID in newSolutions)
hasPossibles

In [ ]:
#casOperons = load(open("pickles/casOperonDataStructureW%s.p" % gene,"rb"))
for protID in missingIDs: 
    if protID not in newSolutions: continue
        
    genomicAsmName = casOperons.revMap[protID] 
    protOperon = casOperons.casOperon[genomicAsmName]
    protCRISPR = protOperon.crisprs[protID]
    if 'pilerCR' in protCRISPR: locus = protCRISPR['pilerCR']
    else:
        locus = protCRISPR['minCED']
        locus.calc_consensus() 
        
    print(protID,locus.consensusRepeat)
    rho = newSolutions[protID]
    for reqID, seq in rho.records.items():
        print(reqID,"\t","%.2f" % CalcPercIdent(locus.consensusRepeat[0],seq),len(seq))
    break

In [ ]:
t1 = rho.terminators[0]

In [ ]:
from easyFunctions import Coordinate
class ErpinOut:
    def __init__(self, outfile="tmp/rhoInd.out", inputfile="tmp/possibleTracrs.fasta"):
        self.numRecords = 0
        self.terminators = []
        self.records={}
        for rec in parse(inputfile,"fasta"):
            self.numRecords += 1
            self.records[rec.id]=str(rec.seq)
        with open(outfile) as file:
            for i in range(9): file.readline()
            capture = False
            for line in file:
                if capture: 
                    line = line.strip().replace("  "," ").replace("  "," ").replace("  "," ")
                    self.terminators.append(RhoIndTerminator(seqName,line.split(" ")))
                capture = (">" in line)
                if capture: seqName = line.strip().replace(">","")
                    
class RhoIndTerminator:
    def __init__(self,name,info):
        print("This it the sequence:",str(info))
        self.name = name
        self.strand = (info[0]=="FW")
        start, end = info[2].split("..")
        self.Rholocation = Coordinate(start,end)
        self.fwd = int(self.name[-1]) % 2 != 0
    def __str__(self): return "%s\t%s\t%s\t%s\t" % (self.name,str(self.strand),str(self.Rholocation),str(self.fwd))

In [ ]:
protID = "NZ_KV814503.1_ORF2438"
conRepeat = "sequences/conRepeats/%s.fasta" % (protID)
write([SeqRecord(id=protID,description='',seq=Seq(locus.consensusRepeat[0]))],conRepeat,"fasta")
chrAsmName = "assemblies/pseudoChromos/%s.fasta" % (protID)
blast_results = parseSingleBLAST(BLAST_short(conRepeat, chrAsmName, "blastout/conRepeats/%s.xml" % (protID)))
boundaryHits,crRNALen = locus.checkArrayBoundaries(blast_results)

chrRec = fasta_index(chrAsmName,"fasta")
chrSequence = str(chrRec[list(chrRec.keys())[0]].seq)
index = 0
terminusSeqs, index = locus.getTerminusRepeats(chrSequence,index)
print("")
print(locus.consensusRepeat)
print("")
for line in terminusSeqs.split("\n"): print(line," ",len(line))
tmpFasta = open("tmp/possibleTracrs.fasta","w")
tmpFasta.write(terminusSeqs)

tmpFasta.close()
cmd = "~/bin/Arnold/erpin ~/bin/Arnold/rho-indep.epn tmp/possibleTracrs.fasta -1,4 -add 1 4 2 -pcw 1 -cutoff 100% >tmp/rhoInd.out"
system(cmd)

##Setp 2j - process the terminators that were found
erpOut = ErpinOut()
if len(erpOut.terminators)==0: 
    print("\nThis ref has nothing: %s\n" % protID)
else: 
    for terminator in erpOut.terminators:
        tracrSeq=""
        if terminator.fwd and not terminator.strand: #Terminator must be on the anti-sense strand
            ############################ANTI-REPEAT
            ############################RC-Anti
            print("Upstream terminator:",str(terminator))
            tracrSeq = s1[terminator.Rholocation.start-1:].upper()
        elif not terminator.fwd and terminator.strand: 
            print("Downstream terminator:",str(terminator))
            tracrSeq = s1[:terminator.Rholocation.end].upper()
        solSize = len(tracrSeq)
        print (tracrSeq,solSize)

## Round 0 - Predictions From Scratch

In [ ]:
s1.find(RC('GCTGGGAATCAATCACCAATCCCCTTTGATATACTG'))

In [ ]:
prpath = "/mnt/research/germs/shane/databases/crisprs/pilerCR/genbank/GCA_003480745.1.pcrout"
cr = PilerCRReader(prpath)['QUJS01000001.1']
print(cr.consensusRepeats)
protID = 'QUJS01000001.1_ORF40881'
conRepeatFP = "sequences/conRepeats/%s.fasta" % (protID)
chrAsmName = "assemblies/pseudoChromos/%s.fasta" % (protID)
print(conRepeatFP)
# print()
conRepeats = open(conRepeatFP,'w')
conRepeats.write(cr.repeatSeqs(protID))
conRepeats.close()
blastResults = BLAST_short(conRepeatFP, chrAsmName, "blastout/conRepeats/%s.xml" % (protID))
blast_results = parseBLAST(blastResults)
for res in blast_results: print(res)

In [ ]:
fh = open("sequences/testRC.fasta","w")
from Bio.SeqIO import parse,write
for rec in parse("sequences/KnownTracrRNAs.fa","fasta"):
    write(rec,fh,"fasta")
    rec.id = rec.id+"RevComp"
    rec.seq = rec.seq.reverse_complement()
    write(rec,fh,"fasta")
fh.close()

In [ ]:
s1

In [ ]:
alignSequence(s1,locus.consensusRepeat[0])

In [ ]:
def RC(seq): return str(Seq(seq).reverse_complement())



In [ ]:
t1 = erpOut.terminators[0]
t2 = erpOut.terminators[1]

In [ ]:
str(t2.Rholocation),len(s1)

In [ ]:
len(locus.consensusRepeat[0])

In [ ]:
t1.Rholocation.end

In [ ]:
t2.fwd

In [ ]:
solSize = 0
tracrSeq = ""
if t1.fwd and not t1.strand: #Terminator must be on the anti-sense strand
    ############################ANTI-REPEAT
    ############################RC-Anti
    tStart = t1.Rholocation.start-1
    tracrSeq = s1[tStart:].upper()
    solSize = len(tracrSeq)
elif not t1.fwd and t1.strand: 
    tracrSeq = s1[:t1.Rholocation.end].upper()
print (tracrSeq)


In [ ]:
%%bash
cat tmp/rhoInd.out

In [ ]:
crispr.terminators = []
crispr.getAntiRepeatCandidates(open("tmp/possibleTracrs.fasta","w"), operon.getSeq())
res = system("~/bin/Arnold/erpin ~/bin/Arnold/rho-indep.epn tmp/possibleTracrs.fasta -1,4 -add 1 4 2 -pcw 3.0 -cutoff 100% >tmp/rhoInd.out")
erpOut = ErpinOut()
len(erpOut.terminators), crispr.getTracrRNA_Candidates(erpOut,open("test.txt",'w'))

In [ ]:
allAsm = fasta_index("" % (id), "fasta")
for id,rec in casRelatedAssemblies.items():
    if path.exists("assemblies/pseudoChromos/%s.fasta" % (id)):continue
    with open("assemblies/pseudoChromos/%s.fasta" % (id),"w") as fh: write(allAsm[id],fh,"fasta")

In [ ]:
def findBiggestCluster(clusters, clusterMap): return Counter(dict((k, clusterMap[k]) for k in (clusters))).most_common(1)[0][0] 

newColors,newColorMap, TreeColors,newIDs = {}, {}, knownTracrRNAs.copy(), set()
startLen = len(TreeColors)
print "Started with %i tracrRNAs" % (startLen)
for seqID in newTracrRNAs:
#     print "Next",seqID
    #Get all the clusters the ID is associated with:
    clusters = infernalResults[seqID]
    
    #Get all of the IDs associated with those clusters
    clusterIDS = set()
    for cluster in clusters: clusterIDS = clusterIDS.union(infernalResults[cluster])
        
    #Check to see if there is overlap betweeb the IDs of the known and the IDs of the predicted for this new tracr
    overlappingIDs = clusterIDS.intersection(knownTracrRNAs)
    
    if len(overlappingIDs)>0:#If there is overlap
        #Get the cluster color(s) of the overlapping IDs
        colors = set()
        for o_id in overlappingIDs: colors.add(knownTracrRNAs[o_id])
            
        #Choose the biggest color
        id_color = findBiggestCluster(colors,knownTracrColorCounter)

    else: #No overlap, choose the best color from new clusters
        biggestCluster = findBiggestCluster(clusters,unknownClusterCounter)
        try: id_color = newColors[biggestCluster]; newColorMap[newColors[biggestCluster]].add(seqID)
        except: id_color = newColors[biggestCluster] = genColor(); newColorMap[newColors[biggestCluster]] = set([seqID])
        newIDs.add(seqID)
    TreeColors[seqID] = id_color

print "Annotated %i/%i" % (len(TreeColors),len(casRelatedAssemblies))
print "\t",len(TreeColors)-startLen, "found"
print "\t",len(newIDs), "/", len(TreeColors)-startLen, "contain a unique structure"
print "\t",len(newColors), "/", len(newColors)+len(knownTracrColorCounter), "new clusters"

dump(TreeColors, "pickles/PredictedTracrRNA_colors.p")

[Home](#annotate)

<a id="ref">reference</a>

In [ ]:
crRNALens, crisprLocations, noPredictedTracr, seqLenDist = [],[],[],[]
hasNRegionSet,crRNALens, possibleSolutions, newSolutions, index, counter = set(),[], set(), {}, 0, 0
allLoci = {}
nCounter, n_and_no_possibles = 0,0
# novelLoci = pickle.load(open("pickles/NovelTracrs.p","rb"))
for protID in casRelatedAssemblies:
#     if protID not in novelLoci: continue
    #Step 1: Select the CRISPR array from the CRISPR Arrays
#     baseID = protID[:protID.find("_ORF")]
    if protID not in pilerCR_results and protID not in minCED_results: 
        die
        continue #Missing from both
    try: 
        if protID in pilerCR_results: assemblyLoci = pilerCR_results[protID].values()
        else: assemblyLoci = pilerCR_results[baseID].values() #Looking only at the most abundant crispr
        for locus in assemblyLoci:
            if locus.name == protID: break
        if locus.name != baseID: die
    except: 
        try: assemblyLoci = minCED_results[baseID].values()
        except: assemblyLoci = minCED_results[protID].values()
                 
        for locus in assemblyLoci:
            if locus.name == baseID: break
        if locus.name != baseID: die

    write([SeqRecord(id=protID,description='',seq=Seq(locus.consensusRepeat[0]))],"tmp/consFasta.fa","fasta")
    blast_results = parseSingleBLAST(BLAST_short("tmp/consFasta.fa", "assemblies/%s.fa" % (protID), "tmp/consBlast.xml"))
    
    #Step 2b
    boundaryHits,crRNALen = locus.checkArrayBoundaries(blast_results)
    crRNALens.append(crRNALen)
    locus.annotate(casRelatedAssemblies[protID], "assemblies/%s.fa" % (protID))
    if locus.hasNRegion: 
        nCounter += 1
        hasNRegionSet.add(protID)
    allLoci[protID] = locus
    
    #Step 2c
    tmpFasta = open("tmp/possibleTracrs.fasta","w")
    
    #Step 2d
    terminusSeqs, index = locus.getTerminusRepeats(casRelatedAssemblies[protID],index)
    tmpFasta.write(terminusSeqs)
    
    #Collect some stats
    startCoord = locus.repeatCoords[0]
    endCoord = locus.repeatCoords[-1]
    seqlen = len(casRelatedAssemblies[protID])
    seqLenDist.append(seqlen)
    minDist = min(seqlen-startCoord.start,seqlen-startCoord.end,seqlen-endCoord.start,seqlen-endCoord.end)
    minDist =  min(minDist,startCoord.start,startCoord.end,endCoord.start,endCoord.end)
    crisprLocations.append(minDist)

    ##Step 2e - Write sequence surounding hits to the fasta file
    for rec in boundaryHits: index = getSurroundingSequence(tmpFasta,index,casRelatedAssemblies[protID],rec,500)
    
    ##Step 2f - Look for terminators in the possible tracrRNAs
    tmpFasta.close()
    cmd = "~/bin/Arnold/erpin ~/bin/Arnold/rho-indep.epn tmp/possibleTracrs.fasta -1,4 -add 1 4 2 -pcw 1 -cutoff 100% >tmp/rhoInd.out"
    system(cmd)
    
    ##Setp 2j - process the terminators that were found
    erpOut = ErpinOut()
    if len(erpOut.terminators)==0: 
        print "\nThis ref has nothing: %s\n" % protID
        noPredictedTracr.append(protID)
        if locus.hasNRegion: n_and_no_possibles +=1
    else: 
        newSolutions[protID] = erpOut
        repeatIndex = 0
    BLAHBLAH
    ## Everything below this point is for getting and annotation a genbank file for the assembly associated with the CRISPR ##
    ##Look through terminators
    for terminator in erpOut.terminators:
        info= terminator.name.split("_")
        whereRepeat = (info[1] == "S")
        start = int(info[-3])
        end = int(info[-2])
        strand = -1
        if terminator.strand == "+": strand = 1
        if whereRepeat: 
            tracr_size = end - (end-min(terminator.Rholocation.start, terminator.Rholocation.end))
            if tracr_size > 350:continue
            repeatFeature = SeqFeature(FeatureLocation(end-min(terminator.Rholocation.start, terminator.Rholocation.end), end), type="CDS", strand=strand)
        else: 
            tracr_size = (start + max(terminator.Rholocation.start, terminator.Rholocation.end)) - start
            if tracr_size > 350:continue
            repeatFeature = SeqFeature(FeatureLocation(start,start + max(terminator.Rholocation.start, terminator.Rholocation.end)), type="CDS", strand=strand)
        fid = "Theoretical tracrRNA_%i" % (repeatIndex)
        repeatFeature.qualifiers['label'] = [fid]
#         locus.orfs.records[baseID].features.append(repeatFeature)
        repeatIndex += 1 
    
    #shift annoations by buffer
# #     if not locus.orfs:continue
#     locus.orfs.records[baseID].seq = locus.orfs.records[baseID].seq[max(locus.min-500,0):locus.max+500]
#     #locus.orfs.records[protID].seq = locus.orfs.records[protID].seq[max(locus.min-500,0):locus.max+500]
#     for rec in locus.orfs.records[baseID].features:
# #         print "This is the start:", rec.location.start - max(locus.min-100,0), rec.location.end - max(locus.min-100,0),rec.location.start,rec.location.end
#         rec.location = FeatureLocation(rec.location.start - max(locus.min-500,0), rec.location.end - max(locus.min-500,0))
# #         print "This is the   end:", rec.location.start, rec.location.end
    
    # Step 2k Create setup for GB file for Vector NTI
#     for id in locus.orfs.records:
#         print "annotations/%s.gb" % (locus.name)
#         fh = open("annotations/%s.gb" % (locus.name),"w")
#         biopythonID = id[:id.find(":")-1] #Biopython limits genbank file ids to 16 characters
#         locus.orfs.records[id].id = biopythonID[:16]
#         locus.orfs.records[id].name = biopythonID[:16]
#         write([locus.orfs.records[id]],fh,'genbank')
#         fh.close()
print (len(allLoci))
pickle.dump(allLoci,open("pickles/TracrLoci.p","wb"))    
pickle.dump(newSolutions,open("pickles/%s_predictedTracrRNAs.p" % (gene),"wb"))
print("\n\nNumber of proteins with large N region and no terminator / with total large N region: %i / %i" % (n_and_no_possibles,nCounter))
print("%i possible solutions in %s references. Nothing found in %i references" % (index-1, len(newSolutions),len(noPredictedTracr)))

In [ ]:
import matplotlib.pyplot as plt
plt.hist(numRepeats)
plt.ylabel('Frequency');
plt.xlabel('Number of Repeats');
plt.title('Repeat Elements in CRISPR Arrays')
plt.savefig('images/%s_CRISPR_Repeats.png' % gene)
plt.show();

In [ ]:
badIDs = set()
for protID in casRelatedAssemblies:
    if protID not in pilerCR_results:
        crisprResults = PilerCRReader("crisprs/%s.pcrout" % (protID))
        baseChrID = protID[:protID.rfind("_")]
        if len(crisprResults) > 1: multipleCRISPRS[protID] = crisprResults
        try: pilerCR_results[protID] = crisprResults[baseChrID]
        except: badIDs.add(protID)
len(badIDs)

In [ ]:
remove=set()
for protID in pilerCR_results:
    if protID not in casRelatedAssemblies:
        print(protID)
        remove.add(protID)
        
for protID in remove: del pilerCR_results[protID]
len(pilerCR_results)
for protID in badIDs: del casRelatedAssemblies[protID]
len(casRelatedAssemblies)
aa_s = open("proteins/%s-Like-clustered_full.faa" % (gene),"w")
fasta = open("assemblies/%s_Representative_Assemblies.fasta" % (gene),"w")
for id,rec in casRelatedAssemblies.items():
    write(rec,aa_s,"fasta")
    protAsm = fasta_index("assemblies/pseudoChromos/%s.fasta" % (id), "fasta")
    write(protAsm[id],fasta,"fasta")
fasta.close()
aa_s.close()

In [ ]:
from Bio.SeqUtils import GC
gcRepeats = []
for protID, crispr in pilerCR_results.items(): gcRepeats.append(GC(crispr.consensusRepeat[0]))

import matplotlib.pyplot as plt
plt.hist(gcRepeats)
plt.ylabel('Frequency');
plt.xlabel('GC Content');
plt.title('GC Content of Consensus Repeat')
plt.savefig('images/%s_CRISPR_GC.png' % gene)
plt.show();

In [ ]:
repeatLens = []
for protID, crispr in pilerCR_results.items(): repeatLens.append(len(crispr.consensusRepeat[0]))

import matplotlib.pyplot as plt
plt.hist(repeatLens)
plt.ylabel('Frequency');
plt.xlabel('Consensus Repeat Length');
plt.title('Length of Consensus Repeat in Cas9 Systems')
plt.savefig('images/%s_CRISPR_Len.png' % gene)
plt.show();

In [ ]:
pilerCR_results = load(open("pickles/%s_pilerCR_results.p" % (gene),"rb"))
assembliesWCrisprs = load(open("pickles/allAssemblyCRISPRs.p","rb"))

### Read and store the results

In [ ]:
for pilerCRFileName in crisprResultFiles:
    protID = pilerCRFileName.replace(".pcrout","").replace("crisprs/","")
    if protID not in casRelatedAssemblies: continue
    mincedFileName = pilerCRFileName.replace("pcrout","mnout")
    noMinced = (stat(mincedFileName).st_size == 0)
    noPiler = (stat(pilerCRFileName).st_size <= 200)
    
    if not noMinced and not noPiler:
        crisprs_in_both +=1
        minCED_results[protID]  = MinCEDReader(mincedFileName)
        pilerCR_results[protID] = PilerCRReader(pilerCRFileName)
        minCED_results[protID].calulate_consensus_seqs()
#         for id,locus in pilercrResults.iteritems():
#             if id in mincedResults:
#                 mincedResults[id].combineResults(locus)
    elif not noMinced:
        crisprs_only_in_minced += 1
        minCED_results[protID]  = MinCEDReader(mincedFileName)
        minCED_results[protID].calulate_consensus_seqs()
    elif not noPiler:
        crisprs_only_in_piler += 1
        pilerCR_results[protID] = PilerCRReader(pilerCRFileName)
    else:
        no_results += 1
        noCRISPR.add(protID)
        #Remove files to ensure integrity of data
        #os.system("rm -f %s %s" % (mincedFileName, pilerCRFileName ))  #TODO Delete this after have run once

crisprsFound = crisprs_in_both + crisprs_only_in_minced + crisprs_only_in_piler
print "Found %i crisprs in %s assemblies" % (crisprsFound,crisprsFound+no_results)
print "\tBoth:", crisprs_in_both
print "\tMinced only:", crisprs_only_in_minced
print "\tPiler only:", crisprs_only_in_piler
#print "\tNo CRISPR:",no_results, len(noCRISPR)
pickle.dump(minCED_results,open("pickles/MinCED_CRISPRS.p","wb"))
pickle.dump(pilerCR_results,open("pickles/PilerCR_CRISPRS.p","wb"))

In [ ]:
minCED_results, pilerCR_results = {},{}
crisprs_in_both, crisprs_only_in_piler, crisprs_only_in_minced, no_results, counter = 0, 0, 0, 0, 0
crisprResultFiles = glob("crisprs/*.pcrout")
noCRISPR = set()

for pilerCRFileName in crisprResultFiles:
    protID = pilerCRFileName.replace(".pcrout","").replace("crisprs/","")
    if protID not in casRelatedAssemblies: continue
    mincedFileName = pilerCRFileName.replace("pcrout","mnout")
    noMinced = (stat(mincedFileName).st_size == 0)
    noPiler = (stat(pilerCRFileName).st_size <= 200)
    
    if not noMinced and not noPiler:
        crisprs_in_both +=1
        minCED_results[protID]  = MinCEDReader(mincedFileName)
        pilerCR_results[protID] = PilerCRReader(pilerCRFileName)
        minCED_results[protID].calulate_consensus_seqs()
#         for id,locus in pilercrResults.iteritems():
#             if id in mincedResults:
#                 mincedResults[id].combineResults(locus)
    elif not noMinced:
        crisprs_only_in_minced += 1
        minCED_results[protID]  = MinCEDReader(mincedFileName)
        minCED_results[protID].calulate_consensus_seqs()
    elif not noPiler:
        crisprs_only_in_piler += 1
        pilerCR_results[protID] = PilerCRReader(pilerCRFileName)
    else:
        no_results += 1
        noCRISPR.add(protID)
        #Remove files to ensure integrity of data
        #os.system("rm -f %s %s" % (mincedFileName, pilerCRFileName ))  #TODO Delete this after have run once

crisprsFound = crisprs_in_both + crisprs_only_in_minced + crisprs_only_in_piler
print "Found %i crisprs in %s assemblies" % (crisprsFound,crisprsFound+no_results)
print "\tBoth:", crisprs_in_both
print "\tMinced only:", crisprs_only_in_minced
print "\tPiler only:", crisprs_only_in_piler
#print "\tNo CRISPR:",no_results, len(noCRISPR)
pickle.dump(minCED_results,open("pickles/MinCED_CRISPRS.p","wb"))
pickle.dump(pilerCR_results,open("pickles/PilerCR_CRISPRS.p","wb"))
#pickle.dump(noCRISPR,open("pickles/noCRISPRS.p","wb"))

In [ ]:
from CRISPRtools import *
minCED_results  = pickle.load(open("pickles/MinCED_CRISPRS.p","rb"))
pilerCR_results = pickle.load(open("pickles/PilerCR_CRISPRS.p","rb"))

In [ ]:
def pickTop(idSet,clustPercs):
    if len(idSet) == 1: return list(idSet)[0]
    maxID, maxAA = 0, 0
    for id in idSet:
        if clustPercs[id][0] > maxAA:
            maxAA = clustPercs[id][0]
            maxID = id
    return maxID
# hmm_parser = load(open("pickles/%s_HMM_Parsing_Results.p" % (gene),"rb"))

In [ ]:
crRNALens, crisprLocations, noPredictedTracr, seqLenDist = [],[],[],[]
hasNRegionSet,crRNALens, possibleSolutions, newSolutions, index, counter = set(),[], set(), {}, 0, 0
allLoci = {}
nCounter, n_and_no_possibles = 0,0
# novelLoci = pickle.load(open("pickles/NovelTracrs.p","rb"))
for protID in casRelatedAssemblies:
#     if protID not in novelLoci: continue
    #Step 1: Select the CRISPR array from the CRISPR Arrays
#     baseID = protID[:protID.find("_ORF")]
    if protID not in pilerCR_results and protID not in minCED_results: 
        die
        continue #Missing from both
    try: 
        if protID in pilerCR_results: assemblyLoci = pilerCR_results[protID].values()
        else: assemblyLoci = pilerCR_results[baseID].values() #Looking only at the most abundant crispr
        for locus in assemblyLoci:
            if locus.name == protID: break
        if locus.name != baseID: die
    except: 
        try: assemblyLoci = minCED_results[baseID].values()
        except: assemblyLoci = minCED_results[protID].values()
                 
        for locus in assemblyLoci:
            if locus.name == baseID: break
        if locus.name != baseID: die

    write([SeqRecord(id=protID,description='',seq=Seq(locus.consensusRepeat[0]))],"tmp/consFasta.fa","fasta")
    blast_results = parseSingleBLAST(BLAST_short("tmp/consFasta.fa", "assemblies/%s.fa" % (protID), "tmp/consBlast.xml"))
    
    #Step 2b
    boundaryHits,crRNALen = locus.checkArrayBoundaries(blast_results)
    crRNALens.append(crRNALen)
    locus.annotate(casRelatedAssemblies[protID], "assemblies/%s.fa" % (protID))
    if locus.hasNRegion: 
        nCounter += 1
        hasNRegionSet.add(protID)
    allLoci[protID] = locus
    
    #Step 2c
    tmpFasta = open("tmp/possibleTracrs.fasta","w")
    
    #Step 2d
    terminusSeqs, index = locus.getTerminusRepeats(casRelatedAssemblies[protID],index)
    tmpFasta.write(terminusSeqs)
    
    #Collect some stats
    startCoord = locus.repeatCoords[0]
    endCoord = locus.repeatCoords[-1]
    seqlen = len(casRelatedAssemblies[protID])
    seqLenDist.append(seqlen)
    minDist = min(seqlen-startCoord.start,seqlen-startCoord.end,seqlen-endCoord.start,seqlen-endCoord.end)
    minDist =  min(minDist,startCoord.start,startCoord.end,endCoord.start,endCoord.end)
    crisprLocations.append(minDist)

    ##Step 2e - Write sequence surounding hits to the fasta file
    for rec in boundaryHits: index = getSurroundingSequence(tmpFasta,index,casRelatedAssemblies[protID],rec,500)
    
    ##Step 2f - Look for terminators in the possible tracrRNAs
    tmpFasta.close()
    cmd = "~/bin/Arnold/erpin ~/bin/Arnold/rho-indep.epn tmp/possibleTracrs.fasta -1,4 -add 1 4 2 -pcw 1 -cutoff 100% >tmp/rhoInd.out"
    system(cmd)
    
    ##Setp 2j - process the terminators that were found
    erpOut = ErpinOut()
    if len(erpOut.terminators)==0: 
        print "\nThis ref has nothing: %s\n" % protID
        noPredictedTracr.append(protID)
        if locus.hasNRegion: n_and_no_possibles +=1
    else: 
        newSolutions[protID] = erpOut
        repeatIndex = 0
    
    ## Everything below this point is for getting and annotation a genbank file for the assembly associated with the CRISPR ##
    ##Look through terminators
    for terminator in erpOut.terminators:
        info= terminator.name.split("_")
        whereRepeat = (info[1] == "S")
        start = int(info[-3])
        end = int(info[-2])
        strand = -1
        if terminator.strand == "+": strand = 1
        if whereRepeat: 
            tracr_size = end - (end-min(terminator.Rholocation.start, terminator.Rholocation.end))
            if tracr_size > 350:continue
            repeatFeature = SeqFeature(FeatureLocation(end-min(terminator.Rholocation.start, terminator.Rholocation.end), end), type="CDS", strand=strand)
        else: 
            tracr_size = (start + max(terminator.Rholocation.start, terminator.Rholocation.end)) - start
            if tracr_size > 350:continue
            repeatFeature = SeqFeature(FeatureLocation(start,start + max(terminator.Rholocation.start, terminator.Rholocation.end)), type="CDS", strand=strand)
        fid = "Theoretical tracrRNA_%i" % (repeatIndex)
        repeatFeature.qualifiers['label'] = [fid]
#         locus.orfs.records[baseID].features.append(repeatFeature)
        repeatIndex += 1 
    
    #shift annoations by buffer
# #     if not locus.orfs:continue
#     locus.orfs.records[baseID].seq = locus.orfs.records[baseID].seq[max(locus.min-500,0):locus.max+500]
#     #locus.orfs.records[protID].seq = locus.orfs.records[protID].seq[max(locus.min-500,0):locus.max+500]
#     for rec in locus.orfs.records[baseID].features:
# #         print "This is the start:", rec.location.start - max(locus.min-100,0), rec.location.end - max(locus.min-100,0),rec.location.start,rec.location.end
#         rec.location = FeatureLocation(rec.location.start - max(locus.min-500,0), rec.location.end - max(locus.min-500,0))
# #         print "This is the   end:", rec.location.start, rec.location.end
    
    # Step 2k Create setup for GB file for Vector NTI
#     for id in locus.orfs.records:
#         print "annotations/%s.gb" % (locus.name)
#         fh = open("annotations/%s.gb" % (locus.name),"w")
#         biopythonID = id[:id.find(":")-1] #Biopython limits genbank file ids to 16 characters
#         locus.orfs.records[id].id = biopythonID[:16]
#         locus.orfs.records[id].name = biopythonID[:16]
#         write([locus.orfs.records[id]],fh,'genbank')
#         fh.close()
print (len(allLoci))
pickle.dump(allLoci,open("pickles/TracrLoci.p","wb"))    
pickle.dump(newSolutions,open("pickles/%s_predictedTracrRNAs.p" % (gene),"wb"))
print("\n\nNumber of proteins with large N region and no terminator / with total large N region: %i / %i" % (n_and_no_possibles,nCounter))
print("%i possible solutions in %s references. Nothing found in %i references" % (index-1, len(newSolutions),len(noPredictedTracr)))

In [ ]:
pilerCR_results, multipleCRISPRS = {}, {}
badIDs,numRepeats,geneIDs = [],[],[]
# crisprResultFiles = glob("crisprs/*.pcrout")
for protID in casRelatedAssemblies:
    crisprResults = PilerCRReader("crisprs/%s.pcrout" % (protID))
    baseChrID = protID[:protID.rfind("_")]
    if len(crisprResults) > 1: multipleCRISPRS[protID] = crisprResults
    try: pilerCR_results[protID] = crisprResults[baseChrID]
    except: badIDs.append(protID); continue
    numRepeats.append(len(pilerCR_results[protID].repeatCoords))
    geneIDs.append(protID)
dump(pilerCR_results,"pickles/%s_pilerCR_results.p" % (gene))
print(len(casRelatedAssemblies)-len(badIDs))
badIDs = set(badIDs)
dump(badIDs,"pickles/badIDs.p")

In [ ]:
numPos, numTerminators, tracrSeqDict = {}, {}, {}
newSolutions = pickle.load(open("pickles/%s_predictedTracrRNAs.p" % (gene),"rb"))
index = 0
sizeDist = []
ignoreSol = 0
possibleSol = open("sequences/%s_predictedTracrRNAs.fasta" % (gene),"w")
for ref,psol in newSolutions.iteritems():
    if ref not in casRelatedAssemblies_NoTracr:continue
    for terminator in psol.terminators:
        seq = psol.records[terminator.name]
        solSize = 0
        if terminator.fwd: 
            solSize = terminator.Rholocation.end
            tracrSeq = seq[:solSize].upper()
        else: 
            solSize = len(seq) - terminator.Rholocation.start-1
            tracrSeq = seq[terminator.Rholocation.start-1:].upper()
        sizeDist.append(solSize)
        if solSize>350 or tracrSeq.count("N")>=4:
            ignoreSol+=1
            continue
        index += 1
        tracrSeqDict["%s_%i" % (ref,index)] = tracrSeq
        possibleSol.write(">%s_%i\n%s\n" % (ref,index,tracrSeq))

possibleSol.close()
print "Ignored %i solutions" % ignoreSol
print "Done", index, "possible solutions"


In [ ]:
from pandas import Series
from matplotlib import pyplot as plt
ser = Series(sizeDist)
plt.hist(ser,bins=20, color='c')
plt.axvline(350, color='b', linestyle='dashed', linewidth=2)
plt.text(95,800,'tracrRNA Length cuttoff  ->')
plt.show()
print ser.describe()

In [ ]:
ser = ser[ser<350]
plt.hist(ser,bins=20, color='c')
plt.title = "Possible tracrRNA length distribution"
plt.show()
print ser.describe()

In [ ]:
ser = Series(crRNALens)
ser = ser [ser > 40]
ax = ser.plot.hist(bins=20,title="Test",normed=True,color='g',edgecolor="k") #,grid=True)
ax.set_xlabel("Number of base pairs in average crRNA in CRISPR")
ax.set_ylabel("% Frequency")
ax
print ser.describe()

In [ ]:
%%bash
cd-hit-est -i sequences/PredictedTracrRNAs.fa -o clusters/PredictedTracrRNAs.grouped.fa -M 0 -d 0 -c .90 -T 0

In [ ]:
clusterID = ""
allClusters, allClusterSeqIDs = {},{}
# print tracrSeqDict.keys()
for line in open("clusters/PredictedTracrRNAs.grouped.fa.clstr"):
    if ">Cluster" in line:
        clusterID = line.strip().replace(">","")
    else:
        sequenceID = line[line.find(">")+1:line.find(".")] 
        seqID = line[line.find(">")+1:line.find("...")] 
        seqID = seqID[:seqID.rfind("_")]
        try:allClusters[clusterID].add(seqID)
        except:allClusters[clusterID]=set([seqID])
        try:allClusterSeqIDs[seqID].add(clusterID)
        except:allClusterSeqIDs[seqID]=set([clusterID])


xs,ys,zs=[],[],[]
# fig = plt.figure()
# ax = fig.add_subplot(111, projection='3d')

#for i in range(2,50):
remove=set()
colors,TreeColors={},{}
clusterDist=[]
for cluster, ids in allClusters.iteritems():
    clusterDist.append(len(ids))
    if len(ids) < 2: remove.add(cluster)
    else:
        colors[cluster]=color()
        for id in ids:TreeColors[id.replace(".","_")] = colors[cluster]
for id in remove: del allClusters[id]

print "\tTotal number of clusters:",len(allClusters)
print "\tNumber of nodes covered:",len(TreeColors)

# xs.append(len(allClusters))
# ys.append(len(TreeColors))
# zs.append(i)
# ax.scatter(ys,xs,zs)    
# # xs.reverse()
# # ys.reverse()
# # plt.plot(xs, ys, 'ro')
# plt.show()

In [ ]:
casRelatedAssemblies = fasta_index("proteins/%s-Like-clustered.faa" % (gene),"fasta")
casRelatedAssemblies = dict(casRelatedAssemblies)
badIDs = load(open("pickles/badIDs.p","rb"))
allClusters = load(open("pickles/%s_ClusteringResults.p" % (gene),'rb'))
allClusterSeqIDs = load(open("pickles/%s_RevClusteringResults.p" % (gene),'rb'))
assembliesWCrisprs = load(open("pickles/allAssemblyW_CRISPRs.p","rb"))
pilerCR_results = load(open("pickles/%s_pilerCR_results.p" % (gene),"rb"))
protAssemblyMap = load(open("pickles/%s_protAssemblyMap.p" % gene,"rb"))
clusterPercs = load(open("pickles/%s_ClusterPercs.p" % (gene),"rb"))
crPath = "/mnt/research/germs/shane/databases/crisprs/pilerCR"
eq = 0
progress = True 
badCounter = len(badIDs) 
rounds = 0 
while badCounter != 0 and progress:
    lastLen = badCounter
    rounds += 1
    replaced,failed = set(),set()
    print("Round %i - %i" % (rounds,len(badIDs)))
    for badID in badIDs:
        idCluster = list(allClusterSeqIDs[badID])[0]
        replacementIDS = allClusters[idCluster]
        if badID in replacementIDS: replacementIDS.remove(badID)
        if len(replacementIDS) > 0 :
            eqID = pickTop(replacementIDS,clusterPercs)
            del clusterPercs[eqID]
            badCounter -= 1
            replaced.add(badID)
            crisprResults = PilerCRReader("crisprs/%s.pcrout" % (eqID))
            baseProtID = eqID[:eqID.rfind("_")]
            try: 
                pilerCR_results[eqID] = crisprResults[baseProtID]
                if (len(crisprResults) > 1): multipleCRISPRS[eqID] = crisprResults
                asmID = protAssemblyMap[eqID]
                asmbly = fasta_index(assembliesWCrisprs[asmID],"fasta")
                asmbly[baseProtID].id = eqID
                casRelatedAssemblies[eqID] = asmbly[baseProtID]
                del casRelatedAssemblies[badID]
                eq += 1
            except: failed.add(eqID)
    badIDs = badIDs.difference(replaced)
    badIDs = badIDs.union(failed)
    progress = (lastLen != badCounter)
dump(pilerCR_results,"pickles/%s_pilerCR_results.p" % (gene))
len(badIDs), len(casRelatedAssemblies), eq, rounds

In [ ]:
clusterDist = Series(clusterDist)
clusterDist.plot.hist();

In [ ]:
dist=[]
for cluster, ids in allClusterSeqIDs.iteritems():
    id_len = len(ids)
    if id_len > 3: dist.append(id_len)
dist = Series(dist)
dist.plot.hist();

In [ ]:
tmpDict = {}
tracrSeqDict = fasta_index("sequences/PredictedTracrRNAs.fa","fasta")
for id,seq in tracrSeqDict.iteritems(): tmpDict[id[:id.rfind("_")]]=str(seq.seq).upper()
tracrSeqDict = tmpDict  
for cluster, seqIDList in allClusters.iteritems():
    clusterFileName = "conseqs1/"+cluster.replace(" ","_")+".fasta"
    fh = open(clusterFileName,"wb")
    clusterStats = []
    for seqID in seqIDList: clusterStats.append(len(tracrSeqDict[seqID]))
    clusterStats = Series(clusterStats)
    keepSeqs = clusterStats[~((clusterStats-clusterStats.mean()).abs() > clusterStats.std())]
    minSeq, maxSeq = keepSeqs.min(), keepSeqs.max()
    index=0
    for seqID in seqIDList: 
        seqLen = len(tracrSeqDict[seqID])
        if seqLen >= minSeq and seqLen <= maxSeq:
            index+=1
            fh.write(">%s\n%s\n" % (seqID,tracrSeqDict[seqID]))
    fh.close()
    if index <= 1:
        os.system("rm %s" % (clusterFileName))
        continue
    print clusterFileName,"(",index,"/",len(seqIDList), ")[%i,%i]" % (minSeq,maxSeq),clusterStats.std() 
    os.system("cd-hit-est -i %s -o %s_cluster -M 0 -d 0 -c .90 -T 0 -sc 1" % (clusterFileName,clusterFileName))
    clusteredResults = processClusterFile("%s_cluster.clstr" % (clusterFileName))
    fh = open(clusterFileName,"wb")
    index = 0 
    for seqID,direction in clusteredResults.filter():
        index +=1
        if direction: fh.write(">%s\n%s\n" % (seqID,str(tracrSeqDict[seqID])))
        else: fh.write(">%s\n%s\n" % (seqID, reverse_complement(tracrSeqDict[seqID])))
    fh.close()
    if index <= 1:
        os.system("rm %s" % (clusterFileName))
    os.system("rm %s_cluster*" % (clusterFileName))

In [ ]:
%%bash
for i in 1 2 3 4 5 6 7 8 9 10
do
    sbatch ../scripts/StructureSearch.sb
done

# Process the infernal results

In [ ]:
infernalResults = ProcessInfernal(1)

# Build a structural results tree uisng a combination of the previous and current search results

<h4>Build color mappers and counters and separate into groups</h4>

In [ ]:
knownTracrRNAs = load(open("pickles/KnownTracrStuctureHitColors.p","rb"))
knownTracrIDColorMap, knownTracrColorCounter = {}, {}
for id,color in knownTracrRNAs.iteritems():
    try:knownTracrIDColorMap[color].add(id)
    except:knownTracrIDColorMap[color]=set([id])
    try:knownTracrColorCounter[color]+=1
    except:knownTracrColorCounter[color]=1
knownTracrColorCounter = Counter(knownTracrColorCounter)
unknownClusterCounter = {}
for struct,ids in infernalResults.structMapping.iteritems(): unknownClusterCounter[struct] = len(ids)

#Tell us about what we are annotating
unknownTracrRNAs = set(casRelatedAssemblies.keys()).difference(knownTracrRNAs)        
print "Known:%i/%i\nUnknown:%i/%i\n\tClusters:%i\n" % (len(knownTracrRNAs),len(casRelatedAssemblies),len(unknownTracrRNAs),len(casRelatedAssemblies),len(knownTracrColorCounter)) 

newTracrRNAs = set(unknownTracrRNAs).intersection(infernalResults)
allFound = set(knownTracrRNAs.keys()).union(infernalResults)
stillMissing = set(casRelatedAssemblies.keys()).difference(allFound)
print "Found: %i/%i\nRemaining: (%i/%i or %i/%i)\n\tClusters: %i" % (len(newTracrRNAs),len(unknownTracrRNAs),len(stillMissing),len(unknownTracrRNAs),len(stillMissing),len(casRelatedAssemblies),len(unknownClusterCounter))

print "TracrRNAs:\n\t%i/%i Found\n\tClusters %i  or less because predicted may not have been picked up in previous search" % (len(knownTracrRNAs)+len(newTracrRNAs),len(casRelatedAssemblies),len(unknownClusterCounter))

<h4>Divide the groups into their new/shared colors</h4>

In [ ]:
def findBiggestCluster(clusters, clusterMap): return Counter(dict((k, clusterMap[k]) for k in (clusters))).most_common(1)[0][0] 

newColors,newColorMap, TreeColors,newIDs = {}, {}, knownTracrRNAs.copy(), set()
startLen = len(TreeColors)
print "Started with %i tracrRNAs" % (startLen)
for seqID in newTracrRNAs:
#     print "Next",seqID
    #Get all the clusters the ID is associated with:
    clusters = infernalResults[seqID]
    
    #Get all of the IDs associated with those clusters
    clusterIDS = set()
    for cluster in clusters: clusterIDS = clusterIDS.union(infernalResults[cluster])
        
    #Check to see if there is overlap betweeb the IDs of the known and the IDs of the predicted for this new tracr
    overlappingIDs = clusterIDS.intersection(knownTracrRNAs)
    
    if len(overlappingIDs)>0:#If there is overlap
        #Get the cluster color(s) of the overlapping IDs
        colors = set()
        for o_id in overlappingIDs: colors.add(knownTracrRNAs[o_id])
            
        #Choose the biggest color
        id_color = findBiggestCluster(colors,knownTracrColorCounter)

    else: #No overlap, choose the best color from new clusters
        biggestCluster = findBiggestCluster(clusters,unknownClusterCounter)
        try: id_color = newColors[biggestCluster]; newColorMap[newColors[biggestCluster]].add(seqID)
        except: id_color = newColors[biggestCluster] = genColor(); newColorMap[newColors[biggestCluster]] = set([seqID])
        newIDs.add(seqID)
    TreeColors[seqID] = id_color

print "Annotated %i/%i" % (len(TreeColors),len(casRelatedAssemblies))
print "\t",len(TreeColors)-startLen, "found"
print "\t",len(newIDs), "/", len(TreeColors)-startLen, "contain a unique structure"
print "\t",len(newColors), "/", len(newColors)+len(knownTracrColorCounter), "new clusters"

dump(TreeColors, "pickles/PredictedTracrRNA_colors.p")

# Stats on what was found

In [ ]:
dist = []
for ids in newColorMap.values(): dist.append(len(ids))
dist = Series(dist)
#print dist.describe()
dist.plot.hist();  
print "Total of %i novel tracrRNA clusters" % (len(newColorMap))
print "Containing %i tracrRNAs" % (dist.sum())

# Visually inspect what results are new and especially those in new clusters

In [ ]:
TreeColors = pickle.load(open("pickles/PaperCombinedTreeColors.p"))
knownBoth, structOnly, blastOnly= set(),set(),set()
for id,color in TreeColors.iteritems():
    if   color == "#70857A": knownBoth.add(id)
    elif color == "#F4B266": structOnly.add(id)
    elif color == "#9B7E46": blastOnly.add(id)
print len(knownBoth), len(structOnly), len(blastOnly),"\n"

print len(knownBoth.intersection(newIDs))
print len(structOnly.intersection(newIDs))
print len(blastOnly.intersection(newIDs)), "That's interesting"

for id in newTracrRNAs: TreeColors[id] = "#0099ff"
dump(TreeColors,"pickles/PredictedTracrRNA_colors_bySearchMethod.p")

# Programming Artifacts Below

In [ ]:
aa_s = open("proteins/%s-Like-clustered_full.faa" % (gene),"w")
fasta = open("assemblies/%s_Representative_Assemblies.fasta" % (gene),"w")
for id,rec in casRelatedAssemblies.items():
    write(rec,aa_s,"fasta")
    protAsm = fasta_index("assemblies/pseudoChromos/%s.fasta" % (id), "fasta")
    write(protAsm[id],fasta,"fasta")
fasta.close()
aa_s.close()

In [ ]:
import itertools
for cluster, ids in allClusters.iteritems():
    if len(ids)<=3:continue
    maxOverlap = 0
    for seq1, seq2 in itertools.combinations(ids, 2):
        maxOverlap = max(len(allClusterSeqIDs[seq1].intersection(allClusterSeqIDs[seq2])),maxOverlap)
    print cluster, "%i/%i" % (maxOverlap, len(ids))

In [ ]:
for cluster, ids in allClusterSeqIDs.iteritems():
    print cluster
    num=0
    for id in ids:
        num+=1
        print "\t%i. %s" % (num,id)

In [ ]:
from Bio.SeqIO import parse, write
casRelatedAssemblies = {}
for id in allCas9s:
    seqID = id[:id.rfind("_")]
    for rec in parse("assemblies/%s.fasta" % id,"fasta"):
        if rec.id == seqID:
#             with open("assemblies/%s.fa" % id, "wb") as fh:
#                 write([rec],fh,"fasta") 
            casRelatedAssemblies[id]=str(rec.seq)
            break

In [ ]:
fh = open("casRelatedAssemblies.fasta","wb")
for id,seq in casRelatedAssemblies.iteritems(): fh.write(">%s\n%s\n" % (id,seq))
fh.close()

In [ ]:
numPos
numTerminators
tracrSeqDict

In [ ]:
newSolutions

In [ ]:
%%bash
mkdir tmp
ls -l 

In [ ]:
################### ARTIFACTS BELOW ###########################
clusterSizes = []
index = 0
colors = {
    0:"#C0C0C0", 1:"#808080",
    2:"#FF0000", 3:"#800000",
    4:"#FFFF00", 5:"#808000",
    6:"#00FF00", 7:"#008000",
    8:"#00FFFF", 9:"#008080",
    10:"#0000FF", 11:"#000080",
    12:"#FF00FF", 13:"#800080",
    14:"#DAF7A6", 15:"#FF5733",
    16:"#C70039", 17:"#900C3F",
    18:"#900C3F", 19:"#900C3F",
    20:"#900C3F", 21:"#900C3F",
    22:"#900C3F", 23:"#900C3F",
    24:"#900C3F", 25:"#900C3F",
    26:"#900C3F", 27:"#900C3F"
}
# for cluster in allClusters: 
#     clusterSizes.append(len(allClusterSeqIDs[cluster]))
#     if len(allClusterSeqIDs[cluster])>=3: 
#         fh = open("data/%s.fa" %(cluster.replace(" ","_")),"w")
#         for id in allClusters[cluster]: 
#             fh.write(">%s\n%s\n" % (id,tracrSeqDict[id]))
#             #TreeColors[id[:id.find("_")]] = colors[index]
#             print id[:id.find("_")],
#         print
#         index += 1
#         fh.close()
#     print "data/%s.fa" % (cluster.replace(" ","_"))
# print "# indices needed:", index-1
# ser = Series(clusterSizes)
# ser.plot.hist()
# print ser.describe() 

In [ ]:
files = os.listdir("./")
for fn in files:
    baseid = fn[:fn.find("_ORF")]+"_orfs.blastout"
    #os.system("mv %s %s" % (fn,baseid))
    

In [ ]:
%%bash
ls


In [ ]:
from Bio.pairwise2 import align
from Bio.pairwise2 import format_alignment
crRNA = RC("GTTGTGATTTGCTTTAA")
tracrRNA = "TTAAAGCAATTCACAATAAGGATTATTCCGATGTGAAAACATTAGGTTGCCTCGTCCTACCATACGGGGCTTTTTTT"

import numpy as np

# bestAln = None
# bestMatch, bestGap = 0,0
# bestScore = 0
# for matchScore in np.arange(0.1, 3.0, 0.1):
#     for gapPenalty in np.arange(0.1, 15.0, 0.1):
#         alignment = align.globalmx(tracrRNA, crRNA,matchScore,-gapPenalty)[0]
#         if alignment[2] > bestScore:
#             bestScore = alignment[2]
#             bestMatch = matchScore
#             bestGap = gapPenalty
#             bestAln = alignment

# print (bestScore,bestGap,bestMatch)
# print(format_alignment(*bestAln))

In [ ]:
print(tracrRNA)
print(crRNA)

In [ ]:
crRNA[-1], tracrRNA[-1]

In [ ]:
from Bio import Align
from Bio.SubsMat import MatrixInfo

aligner = Align.PairwiseAligner()
# aligner.substitution_matrix = MatrixInfo.blosum30
# aligner.mode = 'local'

bestAln = None
bestMatch, bestGap = 0,0
bestOpen, bestExtend= 0,0
bestScore = 0
for matchScore in np.arange(0.5, 10.0, 0.5):
    for mismatchScore in np.arange(0.0, 10.0, 0.5):
        for openPenalty in np.arange(0.1, 10.0, 0.1):
            for extendPenatly in np.arange(0.1, 2.0, 0.1):
                aligner.open_gap_score = -openPenalty
                aligner.extend_gap_score = -extendPenatly
                aligner.match = matchScore
                aligner.mismatch = -mismatchScore
                alignment = aligner.align(tracrRNA, crRNA)
                if alignment.score > bestScore:
                    bestScore = alignment.score
                    bestMatch = matchScore
                    bestGap = -mismatchScore
                    bestOpen = -openPenalty
                    bestExtend = -extendPenatly
                    bestAln = alignment
print("Score = %.1f:" % bestScore)       
print(bestAln[0])
print(bestOpen,bestExtend)
print(bestMatch,bestGap)

In [ ]:
bestAln[0].score

In [ ]:
################################ Custom Functions ################################
from Bio.SubsMat import MatrixInfo as matlist
matrix = matlist.blosum62
gap_open = -8
gap_extend = -.8

def RC(seq): return str(Seq(seq).reverse_complement())
def scoreAlign(alignment):
    ref, frag, score, begin, end = alignment
    matches = 0
    for pos in range(len(ref)):
        if ref[pos] == frag[pos]:matches+=1
    return matches/float(len(frag.replace("-","")))         
def scoreAligns(aln1,aln2):
    score1, score2 = scoreAlign(aln1), scoreAlign(aln2)
    if score1 > score2: return aln1,score1*100
    else: return aln2, score2*100   
    
def alignSequences(refSeq,fragment):
    try: aln1 = pairwise2.align.globalds(refSeq, fragment, matrix, gap_open, gap_extend)[0]
    except: aln1 = None
    try: aln2 = pairwise2.align.globalds(refSeq, RC(fragment), matrix, gap_open, gap_extend)[0]
    except: aln2 = None
    if aln1 == None and aln2 == None: return None,0
    elif aln1 == None: top_aln = aln2
    elif aln2 == None: top_aln = aln1
    else: top_aln,alnScore = scoreAligns(aln1,aln2)
    if alnScore == None: alnScore = 0    
    if top_aln == None:print "Here"
    aln_probe, aln_arms, score, begin, end = top_aln
    return alnScore

def alignSequence(refSeq,fragment):
    aln1 = pairwise2.align.globalds(refSeq, fragment, matrix, gap_open, gap_extend)[0]
    try: aln1 = pairwise2.align.globalds(refSeq, fragment, matrix, gap_open, gap_extend)[0]
    except: aln1 = None
    try: aln2 = pairwise2.align.globalds(refSeq, RC(fragment), matrix, gap_open, gap_extend)[0]
    except: aln2 = None
    if aln1 == None and aln2 == None: return None,0
    elif aln1 == None: top_aln = aln2
    elif aln2 == None: top_aln = aln1
    else: top_aln,alnScore = scoreAligns(aln1,aln2)
    if alnScore == None: alnScore = 0    
    if top_aln == None:print "Here"
    aln_probe, aln_arms, score, begin, end = top_aln
    return '%s\t%% Matching %.2f%%\n\t%s' % (aln_probe, alnScore, aln_arms), alnScore

In [ ]:
from types import MethodType
   
#     Temporary method bindings:
#     locus.clusterBLASTResults = MethodType(clusterBLASTResults,locus)
#     locus.getTracrRNA_Candidates = MethodType(getTracrRNA_Candidates,locus)
#     locus.setName = MethodType(setName,locus)
#     locus.getAntiRepeatCandidates = MethodType(getAntiRepeatCandidates,locus)
#     locus.repeatSeqs = MethodType(repeatSeqs,locus)
#     locus.tracrRNACandidateSeqs = set()

class AntiRepeatCandidate:
    def __init__(self,coord,direction='upstream'):
        self.location = coord
        self.directions = set([direction])
    def addDir(self): self.directions.add('downstream')
    def getSeq(self,chrSeq):
        retSeqs = []
        buffer = 350
        seqName = ">Seq_%i_%i_%s"
        for dir in self.directions:
            if dir == 'upstream':
                start, end = max(self.location.start-buffer,0), self.location.end+1
                retSeqs.append(seqName % (start,end,'U'))
                retSeqs.append(chrSeq[start:end])
            else:
                start, end = self.location.start-1, min(self.location.end+buffer,len(chrSeq)-1)
                retSeqs.append(seqName % (start,end,'D'))
                retSeqs.append(chrSeq[start:end])
        return "\n".join(retSeqs)
    def __hash__(self): return hash(self.location)
    def __str__(self): 
        dirs = "upstream/downstream"
        if len(self.directions) == 1: dirs = self.directions.pop()
        return str(self.location) + " "+dirs
    def __lt__(self,other): return self.location < other.location
    def __gt__(self,other): return self.location > other.location

def clusterBLASTResults(self, blastResults):
    spacerLen = len(max(self.spacers, key=len)) + 10
    blastResults.sort()
#     print("Separation:",spacerLen)
    prevResult = blastResults[0]
    self.antiRepeats[prevResult] = AntiRepeatCandidate(prevResult)
#     print("\t",prevResult, "Start")
    for nextResult in blastResults[1:]:
        if prevResult.distance(nextResult) > spacerLen: 
            self.antiRepeats[nextResult] = AntiRepeatCandidate(nextResult)
            if prevResult in self.antiRepeats: self.antiRepeats[prevResult].addDir()
            else: self.antiRepeats[prevResult] = AntiRepeatCandidate(prevResult,'downstream')
#             print("\t",prevResult,prevResult.distance(nextResult))
#         else: print("\t",nextResult)
        prevResult=nextResult
    if nextResult in self.antiRepeats: self.antiRepeats[nextResult].addDir()
    else:  self.antiRepeats[nextResult] = AntiRepeatCandidate(nextResult,'downstream')
#     print("Found %i possible candidate anti's" % (len(self.antiRepeats)))
#     resList = list(self.antiRepeats.values())
#     resList.sort()
#     for res in resList:
#         print("\t",res)
#     return nextResult,prevResult
def getAntiRepeatCandidates(self,fh,chrSeq):
    for antiRepeat in self.antiRepeats.values(): fh.write(antiRepeat.getSeq(chrSeq)+"\n")
    fh.close()
    
def repeatSeqs(self,fh):
    for index, repeat in enumerate(self.consensusRepeats): fh.write(">%s_%i\n%s\n" % (self.name,index,repeat.upper()))
    fh.close()
def setName(self,name): self.name = name
    
def getTracrRNA_Candidates(self,erpOut,fh):
    if len(erpOut.terminators)==0: 
        print ("\nThis ref has nothing: %s\n" % self.name); return
    for i,terminator in enumerate(erpOut.terminators):
        seq = erpOut.records[terminator.name]
        tracrSeq = ""
        if terminator.upstream and not terminator.strand: tracrSeq = seq[terminator.Rholocation.start-1:].upper()
        elif not terminator.upstream and terminator.strand: tracrSeq = seq[:terminator.Rholocation.end].upper()
        if tracrSeq.count("N")>=4: continue
        if tracrSeq != "":
            self.tracrRNACandidateSeqs.add(tracrSeq)
            fh.write(">%s_%i\n%s\n" % (self.name,i,tracrSeq))

### Run CRISPR predictions to find CRISPR arrays in the genomic assemblies where Cas9s were found

In [ ]:
outdir = "crisprs"
assembly_dir = "assemblies/"
assemblies = glob(assembly_dir+"*.fa")
print "Number of assemblie to run CRISPR array detection on:",len(assemblies)
for assembly in assemblies:

    #Run PilerCR
    if path.exists("%s/%s.pcrout" % (outdir, assembly)):retCode1 = 0
    else: retCode1 = system("pilercr -minid 0.85 -mincons 0.8 -minarray 3 -noinfo -in %s -out %s/%s.pcrout" %(assembly_dir+assembly, outdir, assembly))
        
    #Run minced
    if path.exists("%s/%s.pcrout" % (outdir, assembly)):retCode2 = 32512
    else: retCode2 = system("minced -maxSL 75 --maxRL 75 -minRL 16 -minSL 20 -searchWL 6 %s %s/%s.mnout" % (assembly_dir+assembly, outdir, assembly))
    
    if retCode1 != 0: 
        print "%i\npilercr -minid 0.85 -mincons 0.8 -minarray 3 -noinfo -in %s -out %s/%s.pcrout" %(retCode1,assembly_dir+assembly, outdir, assembly)
        break
    if retCode2 != 32512: 
        print "%i\nminced -maxSL 75 --maxRL 75 -minRL 16 -minSL 20 -searchWL 6 %s %s/%s.mnout" % (retCode2,assembly_dir+assembly, outdir, assembly)
        break

In [ ]:
#     rho_s = RhoTermPredict("tmp/possibleTracrs.fasta","tmp/RhoPredictions",protOperon.assembly)
#     if not rho_s.hasHits: continue

    # Step 7. Read the termination signals
    erpOut = ErpinOut()
    #rhoOut = RhoTermPredictOut()
    erpSols += len(erpOut.terminators)
#     rhoSols += len(rhoOut.terminators)

    # Step 8. Get tracrRNA candidates with rho-ind signals
#     operon.crispr.terminators
    operon.crispr.terminators =set()
    operon.crispr.getTracrRNA_Candidates = MethodType(getTracrRNA_Candidates,operon.crispr)
    operon.crispr.getTracrRNA_Candidates(erpOut,possibleSol)
    #rhoOut.terminators = compareTerms(erpOut,rhoOut)
    numNewTracrs = len(operon.crispr.tracrRNACandidateSeqs)
    if numNewTracrs == 0: noPredictedTracr.add(protID)
    totalSols += numNewTracrs
    if i in breakPoints: print(i,end=' ')

possibleSol.close()
# print("RhoTermPredict Solutions:",rhoSols)
print("\nErpin Solutions:",erpSols)
dump(casOperons, "pickles/%s_Operons.p" % gene)
dump(noPredictedTracr, "pickles/%sWithNoPredictedTracr.p" % (gene))
print("Found %i possible tracr solutions from %i assmeblies" % (totalSols,len(casRelatedAssemblies)-len(noPredictedTracr)))